In [ ]:
# RadioBrief – Final Project Notebook
# Daily Summarizer & Translator (Colab + OpenAI + Drive)
# Revue de presse politique automatisée
# Format d'entrée : PDF ou texte collé
# Sortie : Résumés, traductions, classification par sujet
# Auteur : Sara
# Version : 1.0 – Avril 2025

In [ ]:
# Installing necessary Python packages

# This cell installs the libraries required for the notebook to function.
# It also includes error handling to gracefully manage potential installation failures.

# CELL 2: Getting the necessary tools (Python libraries) ready

# Printing a message to the screen saying we're starting the installation
print("Installing/Upgrading required libraries...")

# We need specific versions of some tools to make sure they work together.
# First, installing the specific 'openai' library (version 1.10.0)
# This library lets our code talk to the OpenAI services (like ChatGPT).
# The '--quiet' part means it won't print too much unnecessary stuff while installing.
print("Installing specific openai version...")
!pip install --upgrade openai==1.10.0 --quiet

# Next, dealing with the 'httpx' library. This helps Python code make web requests.
# The 'openai' library version we just installed needs an older version of 'httpx' (0.27.2) to work correctly.
# Newer versions of 'httpx' cause errors with this specific 'openai' version.
# First, removing any existing version of 'httpx' to avoid conflicts.
# The '-y' means automatically say 'yes' to uninstalling.
print("Force installing compatible httpx version (0.27.2)...")
!pip uninstall -y httpx
# Now, installing the specific version 0.27.2.
!pip install httpx==0.27.2 --quiet

# --- Setting up Error Logging ---
# Sometimes things go wrong during installation.
# This section sets up a way to record any errors in a file named 'package_install_errors.log'.

# Importing Python's built-in tool for logging messages and errors.
import logging
# Importing another tool to help get detailed error information (traceback).
import traceback

# Configuring the logging system.
# - level=logging.ERROR: Only record serious errors.
# - filename="package_install_errors.log": The name of the file to save errors to.
# - filemode="w": 'w' means write mode; it will create a new log file each time, overwriting the old one.
# - format="%(asctime)s - %(levelname)s - %(message)s": How each error message should look (timestamp, error level, the message).
logging.basicConfig(level=logging.ERROR,
                    filename="package_install_errors.log",
                    filemode="w",
                    format="%(asctime)s - %(levelname)s - %(message)s")

# --- Defining a Helper Function for Installing ---
# We need to install several other libraries.
# This function makes it easier by handling the installation command and basic error checking.

# Defining our own function named 'install_package'.
# It takes the package name and optionally a specific version number.
def install_package(package, version=None):
    """
    Installing a Python package with an optional version number.
    Also logging any errors that might happen.
    """
    # Creating the full package name string (e.g., "pandas" or "pdfplumber==0.11.6").
    package_spec = f"{package}=={version}" if version else package
    # Printing which package we are currently installing.
    print(f"Installing {package_spec}...")
    try:
        # Using a trick to capture the output from the installation command.
        # This keeps the notebook output cleaner unless there's an error.
        from IPython.utils import io
        with io.capture_output() as captured:
             # Running the 'pip install' command using the system's command line.
             # '--quiet' again reduces the output messages.
             get_ipython().system(f"pip install {package_spec} --quiet")
        # If the installation command finished without crashing, print a success message.
        print(f"Successfully installed {package_spec}.")
    # Catching any error ('Exception') that might occur during the 'try' block.
    except Exception as e:
        # If an error happened, create a detailed error message.
        error_details = f"Error installing {package_spec}: {e}"
        # Checking if the captured output contains specific error details from pip.
        if captured.stderr:
             error_details += f"\\nPIP Error Output:\\n{captured.stderr}"
        # Recording the detailed error in our log file ('package_install_errors.log').
        logging.error(error_details + f"\\n{traceback.format_exc()}")
        # Printing a message telling the user that an error occurred and where to check the log file.
        print(f" Error installing {package_spec}. Check 'package_install_errors.log'.")


# --- Installing the Other Required Libraries ---
# Now, using our helper function 'install_package' to install the rest of the tools.

# Installing 'pdfplumber' (version 0.11.6): For extracting text from PDF files.
install_package("pdfplumber", "0.11.6")
# Installing 'pypdfium2' (version 4.30.1): Another helper library often used by pdfplumber.
install_package("pypdfium2", "4.30.1")
# Installing 'transformers': A popular library for using AI models (like the one for classifying articles).
install_package("transformers")
# Installing 'torch': The underlying library needed by 'transformers' to run the AI models, especially on GPUs.
install_package("torch")
# Installing 'pandas': A powerful library for working with data tables (like our final CSV file).
install_package("pandas")

# --- Finishing Up ---
# Printing separator lines and final instructions.
print("\\n---")
print(" Libraries installation process finished.")
# Printing a VERY IMPORTANT instruction.
print(" IMPORTANT: You MUST restart the runtime now for changes to take effect!")
# Explaining how to restart the runtime in Google Colab.
print("   Go to the menu: 'Runtime' -> 'Restart runtime'")
# Reminding the user what to do after restarting.
print("   After restarting, run the subsequent cells starting from Cell 3 (Drive Mount).")
print("---")

Installing/Upgrading required libraries...
Installing specific openai version...
Force installing compatible httpx version (0.27.2)...
Found existing installation: httpx 0.27.2
Uninstalling httpx-0.27.2:
  Successfully uninstalled httpx-0.27.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.10.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.
Installing pdfplumber==0.11.6...
Installing pypdfium2==4.30.1...
Installing transformers...
Installing torch...
Installing pandas...
\n---
 Libraries installation process finished.
 IMPORTANT: You MUST restart the runtime now for changes to take effect!
   Go to the menu: 'Runtime' -> 'Restart runtime'
   After restarting, run the subsequent cells starting from Cell 3 (Drive Mount).
---


In [ ]:
# Mounting Google Drive to access/save files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Loading the OpenAI API key from Google Drive (safe way)

# CELL 4: Initializing the ASYNCHRONOUS OpenAI Client (SIMPLIFIED)

# --- Importing necessary libraries ---\n",
from openai import AsyncOpenAI # Importing the asynchronous version
import httpx # Still potentially needed for version checks or other sync calls if any
import os
import pkg_resources # For version checking (Deprecated but used in original code)
import sys
import traceback
import inspect # For inspection (optional now, but doesn't hurt)
import asyncio

print("Starting the process of initializing the asynchronous OpenAI client...")
# ... (Keep the version checks, signature inspection, proxy env var checks if desired for debugging) ...
# ... (Keep the API key reading logic) ...

# --- Reading the API Key from the file ---
# ... (Keep this section as is) ...
api_key_file_path = "/content/drive/MyDrive/RadioBrief/openai_key.txt"
openai_api_key = None
if os.path.exists(os.path.dirname(api_key_file_path)):
    print(f"\\nAttempting reading of API key from: {api_key_file_path}")
    try:
        with open(api_key_file_path, "r") as f:
            openai_api_key = f.read().strip()
        if not openai_api_key:
            print(f"------- Warning: Reading file '{api_key_file_path}' succeeded, but it appears empty.")
            openai_api_key = None
        else:
            print(f"------- Finishing reading API key successfully (key starts with: {openai_api_key[:4]}...).")
    except FileNotFoundError:
        print(f"------- Error finding API key file at '{api_key_file_path}'.")
        openai_api_key = None
    except Exception as e:
        print(f"------- Encountered an unexpected error while reading the API key: {e}")
        openai_api_key = None
else:
     print(f"------- Error accessing directory '{os.path.dirname(api_key_file_path)}'. Is Drive mounted?")
     openai_api_key = None


# --- Initializing the ASYNCHRONOUS OpenAI Client (Simplified) --- <<< MODIFIED SECTION
# Proceeding only if the API key was successfully obtained
async_client = None # Reset variable
if openai_api_key:
    print("\\nProceeding with simplified AsyncOpenAI client initialization...")
    try:
        # Initialize AsyncOpenAI directly with the API key.
        # Let the library handle the default httpx client setup.
        async_client = AsyncOpenAI(
            api_key=openai_api_key
            # No http_client argument passed here
        )
        print("------- Finishing AsyncOpenAI client initialization successfully (using default http client).")

    except Exception as e:
        # Handling errors that occur during the AsyncOpenAI client initialization
        print(f"------- Encountered an error during AsyncOpenAI client initialization: {e}")
        print("\\n--- Detailed Traceback ---\")")
        traceback.print_exc()
        print("--- End Traceback ---\\n")
        async_client = None # Ensuring client is None if initialization fails
else:
    # Skipping initialization if the API key wasn't available
    print("\\nSkipping AsyncOpenAI client initialization because the API key could not be obtained.")


# --- Final Status Check ---
# (Keep this section as is)
if async_client:
    print("\\n🚀 The 'async_client' variable is now ready for making asynchronous API calls.")
else:
    print("\\n------- The 'async_client' variable could not be initialized. Check previous error messages.")

Starting the process of initializing the asynchronous OpenAI client...
\nAttempting reading of API key from: /content/drive/MyDrive/RadioBrief/openai_key.txt
✅ Finishing reading API key successfully (key starts with: sk-p...).
\nProceeding with simplified AsyncOpenAI client initialization...
✅ Finishing AsyncOpenAI client initialization successfully (using default http client).
\n🚀 The 'async_client' variable is now ready for making asynchronous API calls.


<ipython-input-2-f40756d25adc>:9: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources # For version checking (Deprecated but used in original code)


In [ ]:
# Setting the save path and loading or creating the summaries CSV
import pandas as pd
from pathlib import Path

# Defining CSV path
csv_path = "/content/drive/MyDrive/RadioBrief/radio_summaries.csv"

# Checking if the file already exists
if Path(csv_path).exists():
    # Loading existing file
    summaries_df = pd.read_csv(csv_path)
    print(" Loaded existing summary file.")
else:
    # Creating a new dataframe if not exists
    summaries_df = pd.DataFrame(columns=["date", "article", "summary", "arabic_translation", "topic"])
    summaries_df.to_csv(csv_path, index=False)
    print(" Created new summary file.")

 Loaded existing summary file.


In [ ]:
political_keywords = [
    # --- General Politics & Concepts ---
    "Politique", "Géopolitique", "Économie politique", "Relations internationales",
    "Souveraineté", "Frontières", "Sécurité", "Défense", "Libéralisme",
    "Conflit", "Guerre", "Crise", "Instabilité", "Tensions géopolitiques", "Cessez-le-feu", "Trêve",
    "Élections", "Scrutin", "Parité",
    "Diplomatie", "Négociations", "Sommet", "Traité", "Accord",
    "Sanctions", "Embargo", "Coercition économique",
    "Aide humanitaire", "Crise humanitaire", "Droits de l'homme",
    "Terrorisme", "Extrémisme", "Ingérence étrangère",

    # --- French Politics & Foreign Policy ---
    # Keep or remove specific parties/politicians based on your needs
    # "Partis politiques", "Les Républicains", "La France Insoumise", "Parti Socialiste",
    # "Rassemblement National", "Europe Écologie Les Verts", "Renaissance", "Horizons",
    # "Assemblée Nationale", "Sénat", "Matignon", "Bercy", "Élysée",
    # "Emmanuel Macron", "Bruno Retailleau", "Gérald Darmanin", # etc. - Update names as needed
    # "Motion de censure", "ZFE", "OQTF", "Délocalisation",
    # --- Key French Foreign Policy Actors ---
    "Quai d'Orsay", "Ministère des Affaires étrangères", "Ministre des Affaires étrangères", "France", # Added France for explicit mentions

    # --- International Institutions & Law ---
    "ONU", "Nations Unies", "Conseil de sécurité", "Résolution", "Casques bleus",
    "OTAN", "NATO",
    "Union Européenne", "UE", "Bruxelles", "Commission européenne", "Parlement européen",
    "G7", "G20", "OMC", "FMI", "Banque mondiale",
    "Union Africaine", "UA",
    "Cour pénale internationale", "CPI", "Cour internationale de Justice", "CIJ",

    # --- Regions & Countries ---
    "Moyen-Orient", "Proche-Orient",
    "Gaza", "Palestinien", "Palestine", "Cisjordanie", "Jérusalem", "Autorité palestinienne",
    "Israël", "Israélien", "Tel Aviv", # Often used metonymically
    "Liban", "Libanais", "Beyrouth",
    "Syrie", "Syrien", "Damas",
    "Jordanie", "Jordanien", "Amman",
    "Égypte", "Égyptien", "Le Caire",
    "Irak", "Irakien", "Bagdad",
    "Iran", "Iranien", "Téhéran",
    "Arabie Saoudite", "Saoudien", "Riyad",
    "Yémen", "Yéménite", "Sanaa",
    "Qatar", "Qatari", "Doha",
    "Émirats arabes unis", "EAU", "Émirati", "Abou Dhabi", "Dubaï",
    "Turquie", "Turc", "Ankara",
    "Afrique du Nord", "Maghreb",
    "Algérie", "Algérien", "Alger",
    "Tunisie", "Tunisien", "Tunis",
    "Maroc", "Marocain", "Rabat",
    "Libye", "Libyen", "Tripoli",
    "États-Unis", "USA", "Américain", "Washington", "Maison Blanche", "Pentagone", "Département d'État",
    "Chine", "Chinois", "Pékin", "Taïwan",
    "Russie", "Russe", "Moscou", "Kremlin",
    "Ukraine", "Ukrainien", "Kiev",

    # --- Specific Topics ---
    "Migration", "Migrants", "Réfugiés", "Asile", "Immigration", "Immigrés", "Flux migratoires", "Frontière", # Added Frontière again as relevant to migration
    "Guerre commerciale", "Commerce international", "Droit de douane", "Accords commerciaux", "Protectionnisme", "Multilateralisme",
    "Nucléaire", "Prolifération", "AIEA", # Added IAEA
    "Énergie", "Pétrole", "Gaz", "OPEP", # Added OPEC

    # --- Groups & Specific Entities (Examples - Adapt as needed) ---
    "Hamas", "Jihad islamique", # Added Islamic Jihad
    "Hezbollah",
    "Houthis", "Ansar Allah", # Added alternative name
    "Talibans", "Afghanistan",
    "État islamique", "Daech", "EI", "ISIS", # Added variations
    "Al-Qaïda",
    # Add specific leader names if consistently relevant (e.g., Biden, Poutine, Xi Jinping, Netanyahu, Raisi, etc.)
    # "Netanyahu", "Biden", "Poutine", "Xi Jinping", "Ben Gvir", "Gallant" # Example Israeli ministers

    # --- Political Ideologies ---
    "Extrême droite", "Populisme", # Added Populisme
    "Autoritarisme", "Souverainisme", "Nationalisme", "Islamisme"
]

In [ ]:
# Defining a function to summarize an article using GPT-3.5

# # CELL 7: Defining an ASYNCHRONOUS function to summarize an article

import traceback
# Importing logging again within the cell in case of partial runs (good practice)
import logging
# Importing the necessary async client type for type hinting (optional but good)
from openai import AsyncOpenAI

# Defining the function as asynchronous using 'async def'
async def async_summarize_article(
    async_client: AsyncOpenAI, # Accepting the async client as an argument
    text: str,
    max_lines: int = 4,
    style: str = "journalistique radio",
    tone: str = "neutre et informatif",
    focus: str = "faits politiques"
) -> str: # Added return type hint
    """
    Asynchronously summarizes an article using the provided AsyncOpenAI client.

    Args:
        async_client: The initialized AsyncOpenAI client instance.
        text: The text of the article to summarize.
        max_lines: The maximum number of lines for the summary.
        style: The journalistic style to adopt (e.g., "journalistique radio").
        tone: The tone of the summary (e.g., "neutral and informative").
        focus: The main focus of the summary (e.g., "political facts").

    Returns:
        A summarized version of the input text, or an error message string.
    """

    # Preparing the prompt
    prompt = f"""
    Résume cet article en {max_lines} lignes maximum, dans un style {style},
    en mettant l'accent sur les {focus}. Utilise un ton {tone}.

    Texte de l'article :
    {text}
    """

    print(f"--> Preparing async summary request for article snippet: {text[:50]}...") # Added print for feedback

    try:
        # Making the API call asynchronous using 'await'
        # Calling the 'create' method on the passed 'async_client'
        response = await async_client.chat.completions.create(
            model="gpt-3.5-turbo", # Using the specified model
            messages=[{"role": "user", "content": prompt}], # Passing the prompt
            temperature=0.3, # Setting the desired temperature
        )

        # Extracting the content
        summary = response.choices[0].message.content.strip()
        print(f"--> Received async summary for article snippet: {text[:50]}...") # Added print for feedback
        return summary

    except Exception as e:
        # Handling potential errors during the asynchronous API call
        logging.error(f"Error summarizing article asynchronously: {e}\n{traceback.format_exc()}\nText: {text[:200]}")
        print(f"❌ Error during async summary request for article snippet: {text[:50]}... Error: {e}") # Added print for feedback
        # Returning a fallback error message
        return "Résumé non disponible."

# --- Example Usage (Commented out - this function is now async and needs to be called differently) ---
# We cannot call an 'async def' function directly like a normal function.
# It needs to be 'awaited' from within another 'async def' function or run using asyncio.run().
# We will handle calling this later in the modified process_articles_from_text function.

# # Example (requires async context to run):
# async def run_example():
#    if async_client: # Make sure async_client is initialized from Cell 4
#        sample_text = "This is a short sample article text for testing the async summarizer."
#        summary = await async_summarize_article(async_client, sample_text)
#        print("\n--- Async Summary Example ---")
#        print(summary)
#
# # To run the example in Colab/Jupyter (requires nest_asyncio if running in notebook):
# # import nest_asyncio
# # nest_asyncio.apply()
# # asyncio.run(run_example())

In [ ]:
# Defining a function to translate a summary into Arabic
# Translating text to Arabic using GPT-3.5 Turbo

# CELL 8: Defining an ASYNCHRONOUS function to translate text to Arabic

# Importing necessary libraries/modules again for clarity and potential partial runs
import traceback
import logging
from openai import AsyncOpenAI # For type hinting

# Defining the function as asynchronous using 'async def'
async def async_translate_to_arabic(
    async_client: AsyncOpenAI, # Accepting the async client as an argument
    text: str,
    style: str = "journalistique",
    formality: str = "formel",
    target_audience: str = "public général",
    context: str = "actualités"
) -> str: # Added return type hint
    """
    Asynchronously translates text into Modern Standard Arabic using the provided AsyncOpenAI client.

    Args:
        async_client: The initialized AsyncOpenAI client instance.
        text (str): The text to be translated.
        style (str, optional): Desired writing style. Defaults to "journalistique".
        formality (str, optional): Degree of formality. Defaults to "formel".
        target_audience (str, optional): Target audience. Defaults to "public général".
        context (str, optional): Context of use. Defaults to "actualités".

    Returns:
        str: The translated Arabic text, or an error message string.
    """

    # Preparing the prompt
    prompt = f"""
    Traduis ce texte en arabe standard moderne (MSA),
    avec une précision élevée et un style {style},
    adapté à un public {target_audience} dans un contexte de {context}.
    Utilise un registre {formality}.
    Conserve la structure et le sens du texte original.

    Texte à traduire :
    {text}
    """

    print(f"--> Preparing async translation request for text snippet: {text[:50]}...") # Added print for feedback

    # Handling the case where the input text might be the error message from summarization
    if text == "Résumé non disponible.":
         print("--> Skipping translation because input text is 'Résumé non disponible.'")
         return "Traduction non disponible car le résumé initial n'était pas disponible."

    try:
        # Making the API call asynchronous using 'await'
        # Calling the 'create' method on the passed 'async_client'
        response = await async_client.chat.completions.create(
            model="gpt-3.5-turbo", # Using the specified model
            messages=[{"role": "user", "content": prompt}], # Passing the prompt
            temperature=0.3, # Setting the desired temperature for translation
        )

        # Extracting the content
        translation = response.choices[0].message.content.strip()
        print(f"--> Received async translation for text snippet: {text[:50]}...") # Added print for feedback
        return translation

    except Exception as e:
        # Handling potential errors during the asynchronous API call
        logging.error(f"Error translating text asynchronously: {e}\n{traceback.format_exc()}\nText: {text[:200]}")
        print(f"❌ Error during async translation request for text snippet: {text[:50]}... Error: {e}") # Added print for feedback
        # Returning a fallback error message
        return "Traduction non disponible."

# --- Example Usage (Commented out - requires async context) ---
# async def run_translation_example():
#    if async_client: # Make sure async_client is initialized from Cell 4
#        sample_summary = "The president announced new economic measures today."
#        translation = await async_translate_to_arabic(async_client, sample_summary)
#        print("\n--- Async Translation Example ---")
#        print(translation)
#
# # To run the example in Colab/Jupyter:
# # import nest_asyncio
# # nest_asyncio.apply()
# # asyncio.run(run_translation_example())

In [ ]:
# CELL 9: Classifier Model Setup & CORRECTED Classification Function Definition

# Importing the 'torch' library, often used for AI models
import torch
# Importing the 'pipeline' tool from the 'transformers' library (Hugging Face)
from transformers import pipeline
# Importing traceback for detailed error messages
import traceback
# Importing logging for recording errors
import logging

print("--- Setting up Cell 9 ---") # Added print

# --- Configuration for Classification (Can still be changed here) ---

# Defining the possible topic labels the AI model should look for.
candidate_labels = ["Politique", "Gouvernement", "Élections", "Diplomatie", "Relations Internationales", "Économie", "Société"]
print(f"Using candidate labels: {candidate_labels}")

# Setting the minimum confidence score (between 0.0 and 1.0) needed to accept a label.
classification_threshold = 0.3
print(f"Classification threshold set to: {classification_threshold}")


# --- Setting up the AI Model ---
# Declaring classifier variable as None initially
classifier = None
try:
    # Checking if a powerful GPU is available for faster processing, otherwise using the CPU
    device_id = 0 if torch.cuda.is_available() else -1
    device_name = 'GPU 0' if device_id == 0 else 'CPU'
    print(f"Attempting to use device: {device_name} for classification pipeline...")

    # Creating the classification AI model (pipeline).
    # Using the "zero-shot-classification" type.
    # Model name: "facebook/bart-large-mnli"
    # Telling it which device to run on (GPU or CPU)
    classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device_id)
    print(f"Classifier pipeline created successfully on {device_name}.")

except Exception as e:
     print(f"❌ Error loading text classification model: {e}")
     print(f"❌ Classification will likely fail later!")
     # Ensure classifier is None if loading fails
     classifier = None


# --- Defining the Classification Function (CORRECTED WITH INPUT CHECK) ---

def classify_topic(pipeline_obj, text_to_classify, labels_list):
    """
    Classifying the article topic using the passed AI model pipeline.
    Includes input validation.

    Args:
        pipeline_obj: The loaded Hugging Face pipeline object for classification.
        text_to_classify (str): The full article text or summary to classify.
        labels_list (list): The list of possible topics (candidate labels).

    Returns:
        dict: The full result dictionary from the pipeline, or an error/skipped indicator.
    """
    # Checking if the passed pipeline object is valid
    if not pipeline_obj:
        logging.error("Classifier pipeline object is missing or invalid!")
        print("❌ ERROR inside classify_topic: Pipeline object missing!")
        # Return a dict matching expected structure but indicating error
        return {'labels': ['Classifier Error'], 'scores': [0.0], 'sequence': str(text_to_classify)[:50]} # Use str() for safety

    # --- ADDED INPUT VALIDATION ---
    # Check if the input text is valid (not None, is a string, and not too short after stripping whitespace)
    if not text_to_classify or not isinstance(text_to_classify, str) or len(text_to_classify.strip()) < 20:
        print(f"(!) Skipping classification for invalid/short input text: '{str(text_to_classify)[:50]}...'")
        # Return a dictionary indicating why it was skipped
        return {'labels': ['Classification Skipped'], 'scores': [0.0], 'sequence': str(text_to_classify)[:50]}
    # --- END OF ADDED INPUT VALIDATION ---

    try:
        # Limit text length for stability (optional but recommended)
        max_length = 512 # Example limit, adjust if needed

        # Running the classification model using the passed pipeline object
        print(f"  (Classifying text starting with: {text_to_classify[:max_length][:50]}...)") # Show beginning of text being classified

        # Using the pipeline_obj that was passed into the function
        # Keeping multi_label=False as it was in the original failing call trace
        result = pipeline_obj(text_to_classify[:max_length], labels_list, multi_label=False) # Apply max_length limit here

        # Getting the label and score for feedback printing (optional but helpful)
        most_likely_label = result['labels'][0]
        confidence = result['scores'][0]
        print(f"  (Raw classification: Label='{most_likely_label}', Score={confidence:.4f})") # Adding feedback

        # Return the entire result dictionary (Cell 12 handles threshold logic)
        return result

    except Exception as e:
        # If something goes wrong during classification, logging the error
        error_text_snippet = str(text_to_classify)[:200] # Ensure snippet is string
        logging.error(f"Error classifying text: {e}\\n{traceback.format_exc()}\\nText: {error_text_snippet}")
        print(f"❌ ERROR inside classify_topic during pipeline call: {e} for text: '{error_text_snippet[:50]}...'") # Added print
        # Return a dict matching expected structure but indicating error
        return {'labels': ['Classification Error'], 'scores': [0.0], 'sequence': error_text_snippet[:50]}



# --- Example Usage (Optional - Should now work correctly if run here) ---
# Checking if classifier loaded before running example
if classifier:
    print("\\n--- Running Example Test within Cell 9 ---")
    sample_text = "Le premier ministre a annoncé une réforme majeure des retraites hier soir lors de son allocution télévisée."
    # Pass the classifier object created above
    topic_dict = classify_topic(classifier, sample_text, candidate_labels)
    print(f"Example Test Result Dict: {topic_dict}")
    # You can optionally print just the top label/score like before
    # print(f"Example Test - Top Label: {topic_dict['labels'][0]}, Score: {topic_dict['scores'][0]:.4f}")
    print("--- Finished Example Test ---")
else:
    print("\\n--- Skipping Example Test (Classifier not loaded) ---")


print("--- Finished setting up Cell 9 ---") # Added print

--- Setting up Cell 9 ---
Using candidate labels: ['Politique', 'Gouvernement', 'Élections', 'Diplomatie', 'Relations Internationales', 'Économie', 'Société']
Classification threshold set to: 0.3
Attempting to use device: GPU 0 for classification pipeline...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


Classifier pipeline created successfully on GPU 0.
\n--- Running Example Test within Cell 9 ---
  (Classifying text starting with: Le premier ministre a annoncé une réforme majeure ...)
  (Raw classification: Label='Politique', Score=0.4019)
Example Test Result Dict: {'sequence': 'Le premier ministre a annoncé une réforme majeure des retraites hier soir lors de son allocution télévisée.', 'labels': ['Politique', 'Gouvernement', 'Société', 'Relations Internationales', 'Diplomatie', 'Économie', 'Élections'], 'scores': [0.4019125998020172, 0.2188386619091034, 0.10287947952747345, 0.08649303764104843, 0.07917815446853638, 0.07063920795917511, 0.040058769285678864]}
--- Finished Example Test ---
--- Finished setting up Cell 9 ---


In [ ]:
# CELL 10
# Defining a function to extract text from an uploaded PDF file
# Extracting text from PDF using pdfplumber

# Importing the necessary libraries
import pdfplumber  # Library specialized for reading PDFs cleanly

# Defining the function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    """
    Extracts all the text from a PDF file using pdfplumber.

    Args:
        pdf_path: The path (location) of the PDF file on your computer or Google Drive.

    Returns:
        The full extracted text from the PDF as a string.
        Returns None if there is an error during extraction.
    """

    # Start with an empty text
    full_text = ""

    try:
        # Open the PDF file using pdfplumber
        with pdfplumber.open(pdf_path) as pdf:
            # Loop through each page one by one
            for page in pdf.pages:
                # Extract the text from the current page
                text = page.extract_text()

                # If there is text on this page, add it to the full_text
                if text:
                    full_text += text + "\n"  # Add a line break between pages

    # Handle the case where the file does not exist
    except FileNotFoundError:
        logging.error(f"PDF file not found: {pdf_path}")
        return None

    # Handle PDF files that are corrupted or have a syntax error
    except pdfplumber.PDFSyntaxError:
        logging.error(f"PDF syntax error in file: {pdf_path}")
        return None

    # Handle any other unexpected error
    except Exception as e:
        logging.error(f"Error extracting text from PDF: {e}\n{traceback.format_exc()}")
        return None

    # If everything goes well, return the extracted full text
    return full_text

In [ ]:
# Smart Split Articles

# CELL 11: Function for Splitting Articles (Flexible Patterns + Dummies Comments)

# Importing the 're' library, which helps Python understand special text patterns (regex)
import re

# Defining our function for splitting the newspaper text
def smart_split_articles(full_text):
    """
    Splitting the newspaper text into articles using specific rules.
    Trying to be smarter about finding where articles start and end.
    """
    # Printing a message to know we are starting the debug process inside this function
    print("--- Debugging smart_split_articles ---")

    # --- First Step: Cleaning up the text ---
    # Getting rid of extra blank lines by replacing multiple newlines ('\n') with just one.
    cleaned_text = re.sub(r'\\n+', '\\n', full_text)
    # Removing a weird character ('\x0c') that sometimes comes from PDFs and replacing it with a newline.
    cleaned_text = cleaned_text.replace('\\x0c', '\\n')
    # Printing the first part of the cleaned text, just for checking
    print("Cleaned text (first 500 chars):")
    print(cleaned_text[:500])
    print("-" * 20) # Printing a separator line

    # --- Second Step: Finding potential places to split based on section titles ---

    # Listing words or phrases often used as section titles in newspapers.
    # We'll look for these words to guess where a new section (and maybe a new article) begins.
    section_keywords = [
        'FRANCE', 'INTERNATIONAL', 'AFRIQUE', 'MOYEN-ORIENT',
        'DIPLOMATIE', 'ALGÉRIE', 'TUNISIE', 'MAROC', 'LIBAN',
        'PALESTINE', 'IRAN', 'MIGRATION', 'ISRAËL', 'UKRAINE',
        'MOYEN ORIENT', 'AFRIQUE DU NORD',
        'SPÉCIAL DROITS DE DOUANE', 'POLICE-JUSTICE',
        'TRANSPORTS', 'FAITS DIVERS', 'ÉCONOMIE',
        'CULTURE & LOISIRS', 'SPORTS', 'HIPPISME'
    ]

    # Creating a smarter search pattern for the section titles.
    # This pattern looks for the keywords from the list above...
    # ...allowing for spaces or newlines before or after the keyword ('\s*')...
    # ...making sure it matches the whole word ('\b')...
    # ...and handling any special characters in the keywords ('re.escape').
    # The '|' means 'OR', so it searches for any keyword in the list.
    split_pattern_flexible = r'\b(?:' + '|'.join(r'\s*' + re.escape(kw) + r'\s*' for kw in section_keywords) + r')\b'

    # Splitting the cleaned text wherever one of our flexible section title patterns is found.
    # Using 'flags=re.IGNORECASE' means it doesn't matter if titles are uppercase or lowercase.
    # This should give us chunks of text that are hopefully between the section titles.
    possible_articles = re.split(split_pattern_flexible, cleaned_text, flags=re.IGNORECASE)
    # Printing how many chunks we got after this first split, for checking.
    print(f"Number of chunks after splitting by FLEXIBLE section headers: {len(possible_articles)}")
    print("-" * 20) # Printing a separator line

    # --- Third Step: Trying to split the chunks further using ALL CAPS lines (potential headlines) ---
    final_articles = [] # Creating an empty list to hold the final articles
    # Looping through the chunks we got from the section title split
    # Using 'enumerate' gives us an index 'i' (0, 1, 2...) for each chunk, useful for printing debug info
    for i, article_chunk in enumerate(possible_articles):
        # Checking if the chunk actually contains text (it might be empty if two headers were back-to-back)
        if article_chunk and article_chunk.strip():

           # --- MODIFIED: Process the whole chunk without ALL CAPS split ---
            trimmed_chunk = article_chunk.strip() # Trim the original chunk directly
            # Checking if the original chunk is long enough and contains letters
            if len(trimmed_chunk) > 150 and any(c.isalpha() for c in trimmed_chunk):
                 # Add the original chunk if it meets criteria
                 final_articles.append(trimmed_chunk)
            # --- END MODIFICATION ---

        else:
            # Printing a message if the chunk from the first split was empty
             print(f"  Chunk {i}: Skipped (was empty after first split).")

    print("-" * 20) # Printing a separator line
    # Printing the final count of articles we collected after all splitting and filtering
    print(f"  (Splitting attempt resulted in {len(final_articles)} potential articles after filtering)")
    print("--- Finished debugging smart_split_articles ---") # Printing a message to know the function is done
    # Returning the list of potential articles found
    return final_articles

# === Test Call (Optional - Keeping it here for now for checking) ===
# This part tries to run the function immediately with the 'pdf_text' variable
# to see how many articles it finds directly within this cell's execution.
# if 'pdf_text' in locals() and pdf_text:
#     print("\\n--- Running Test Call within Cell 11 ---")
#     test_split = smart_split_articles(pdf_text) # Calling the function
#     print(f"Test call returned {len(test_split)} articles.") # Printing the result count
# else:
#     # Printing a warning if the 'pdf_text' variable isn't available
#     print("\\nVariable 'pdf_text' not found. Cannot run test call in Cell 11.")

In [ ]:
# Defining a function to split full text and process each article
# CELL 12: Defining an ASYNCHRONOUS function to process articles (MODIFIED with keyword density)

# Importing necessary libraries/modules
from datetime import datetime
import asyncio # For asynchronous operations
import pandas as pd
import logging
import traceback
from openai import AsyncOpenAI # For type hinting
import re # Needed for keyword searching

# --- Configuration for Keyword Density (Easy to change here!) ---
# Setting the minimum number of times a keyword must appear in an article
# for it to be considered political based *only* on keywords.
min_keyword_hits = 3 # Try changing this number (e.g., 2, 4, 5)

# --- Defining the ASYNCHRONOUS processing function ---
async def async_process_articles_from_text(
    async_client: AsyncOpenAI, # Accepting the async client
    full_text: str,
    political_keywords: list, # Accepting keywords list from Cell 6
    summaries_df: pd.DataFrame, # Accepting the dataframe to modify
    csv_path: str # Accepting the path to save CSV
) -> pd.DataFrame: # Returning the modified dataframe
    """
    Asynchronously processing articles from full text:
    - Splitting text into articles (using function from Cell 11).
    - Classifying topics OR checking for a minimum number of political keywords (keyword density). # <<< MODIFIED description
    - Creating batches of summarization and translation tasks for political articles.
    - Running API calls concurrently using asyncio.gather.
    - Saving results to the DataFrame and CSV.
    """
    global classifier # Using the classifier model defined in Cell 9
    global candidate_labels # Using the labels defined in Cell 9

    print("\\nStarting asynchronous article processing...")
    # Step 1: Splitting the full text into potential articles using our function from Cell 11
    articles = smart_split_articles(full_text)
    print(f"Split text into {len(articles)} potential articles.")

    # --- Preparation Phase: Identifying tasks ---
    political_articles_data = [] # Storing data for articles identified as political
    non_political_skipped = 0
    too_short_skipped = 0

    # Compiling keywords into a single efficient regex pattern for searching
    # Using word boundaries (\\b) and making it case-insensitive (?i)
    keyword_pattern = r'(?i)\\b(?:' + '|'.join(re.escape(kw) for kw in political_keywords) + r')\\b'
    keyword_regex = re.compile(keyword_pattern)
    print(f"Compiled {len(political_keywords)} keywords into regex pattern.")

    print(f"Identifying political articles (using classifier OR >= {min_keyword_hits} keyword hits)...") # <<< MODIFIED print
    # Looping through each potential article found
    for i, article_text in enumerate(articles):
        # Removing extra whitespace from the start and end
        article_text = article_text.strip()

        # Skipping articles that are too short or seem empty
        # if len(article_text) < 300 or not any(c.isalpha() for c in article_text):
        #     too_short_skipped += 1
        #     continue

        # --- MODIFIED Identification Logic (Classifier + Keyword Density Fallback) ---
        is_political = False
        assigned_topic = "Non Politique" # Starting assumption

        # Step 4a: Trying to classify using the AI model first (function from Cell 9)
        # This now uses the full article text
        topic_result = classify_topic(classifier, article_text, candidate_labels)
        print("topic_result")
        print(topic_result)
        print("topic_result")
        most_likely_label = topic_result['labels'][0] # Using the specific label found
        confidence = topic_result['scores'][0]

        # Checking if the classifier found a confident political topic
        if most_likely_label != "Non Politique (Below Threshold)" and most_likely_label != "Classification Error" and confidence >= classification_threshold:
             is_political = True
             assigned_topic = topic_result['labels'][0] # Using the specific topic found ------------------------- modified to Get only the top label string so cell 15 would work
            #  assigned_topic = topic_result # Using the specific topic found ------------------------- modified to Get only the top label string so cell 15 would work
             print(f"  -> Article {i+1} identified as '{assigned_topic}' by classifier.")
        # else: # Optional: print if classifier deemed non-political or failed
             # print(f"  -> Article {i+1} classified as '{topic_result}'. Checking keywords...")

        # Step 4b: If the classifier didn't confidently identify it, checking for keyword density
        if not is_political and most_likely_label != "Classification Error":
            # Using 're.findall' to find ALL occurrences of any keyword in the full article text
            keyword_matches = keyword_regex.findall(article_text)
            # Counting how many keywords were found
            number_of_hits = len(keyword_matches)

            # Checking if the number of hits meets our minimum requirement (min_keyword_hits)
            if number_of_hits >= min_keyword_hits:
                 is_political = True
                 # Assigning a generic topic, as keywords don't give a specific topic like the classifier
                 assigned_topic = f"Politique ({number_of_hits} Keyword Hits)"
                 print(f"  -> Article {i+1} identified as '{assigned_topic}' (met threshold of {min_keyword_hits}).")
            else: # Optional: print if keyword density check also failed
                 print(f"  -> Article {i+1} had only {number_of_hits} keyword hits (less than {min_keyword_hits}).")

        # --- End MODIFIED Identification Logic ---

        # Step 5: Skipping if neither classifier nor keyword density marked it as political
        if not is_political:
            non_political_skipped += 1
            continue
        else:
            # Storing the data for political articles that passed the checks
            political_articles_data.append(
                {"index": i, "text": article_text, "topic": assigned_topic} # Storing the assigned topic
            )

    print(f"Identified {len(political_articles_data)} political articles (via classifier or keyword density).") # <<< MODIFIED print
    print(f"Skipped {non_political_skipped} non-political articles.")
    print(f"Skipped {too_short_skipped} short/empty articles.")

    # --- Execution Phase: Running Summaries and Translations (No changes needed below this point) ---
    # The rest of this function (creating and running summary tasks,
    # creating and running translation tasks, saving results) remains the same as before.
    # It will only run these tasks on the articles collected in 'political_articles_data'.

    summary_tasks = []
    original_indices_for_summary = [] # Keep track of original index for mapping results

    # Step 6a: Create summary tasks
    print(f"\\nCreating {len(political_articles_data)} summary tasks...")
    for article_data in political_articles_data:
         # Handling long articles by chunking (same as before)
         if len(article_data["text"]) > 12000:
             print(f"  -> Article {article_data['index']+1} is long, preparing chunked summary task (sync chunking)...\")")
             chunk_size = 3000
             chunks = [article_data["text"][j:j+chunk_size] for j in range(0, len(article_data["text"]), chunk_size)]
             async def summarize_long_article_chunks(chunks_list):
                 chunk_tasks = [asyncio.create_task(async_summarize_article(async_client, chunk)) for chunk in chunks_list]
                 chunk_summaries = await asyncio.gather(*chunk_tasks, return_exceptions=True)
                 processed_summaries = [str(s) if not isinstance(s, Exception) else "Résumé Chunk Error." for s in chunk_summaries]
                 return " ".join(processed_summaries)
             summary_tasks.append(asyncio.create_task(summarize_long_article_chunks(chunks)))
             original_indices_for_summary.append(article_data["index"])
         else:
             # Creating a task for the async_summarize_article function (from Cell 7)
             task = asyncio.create_task(
                 async_summarize_article(async_client, article_data["text"])
             )
             summary_tasks.append(task)
             original_indices_for_summary.append(article_data["index"]) # Store corresponding index

    # Step 6b: Running summary tasks concurrently using asyncio.gather
    print(f"Running {len(summary_tasks)} summary tasks in parallel...")
    summary_results = await asyncio.gather(*summary_tasks, return_exceptions=True)
    print("Finished running summary tasks.")

    # --- Processing Summaries and Creating Translation Tasks (same as before) ---
    translation_tasks = []
    original_indices_for_translation = []
    summaries_for_translation = {}

    print("\\nProcessing summary results and creating translation tasks...")
    for i, result in enumerate(summary_results):
        original_index = original_indices_for_summary[i]
        article_data = next(item for item in political_articles_data if item["index"] == original_index)
        if isinstance(result, Exception):
            summary = "Résumé non disponible (Erreur)."
            print(f"  -> Summary failed for article {original_index+1}: {result}")
            logging.error(f"Async summary failed for article {original_index+1}: {result}")
            summaries_for_translation[original_index] = summary
        elif result == "Résumé non disponible.":
             summary = result
             print(f"  -> Summary unavailable for article {original_index+1} (from function).")
             summaries_for_translation[original_index] = summary
        else:
            summary = str(result)
            summaries_for_translation[original_index] = summary
            # Creating translation task ONLY if summary was successful (function from Cell 8)
            task = asyncio.create_task(
                async_translate_to_arabic(async_client, summary)
            )
            translation_tasks.append(task)
            original_indices_for_translation.append(original_index)

    # Step 7b: Running translation tasks concurrently
    print(f"Running {len(translation_tasks)} translation tasks in parallel...")
    translation_results = await asyncio.gather(*translation_tasks, return_exceptions=True)
    print("Finished running translation tasks.")

    # --- Finalizing: Combining results and saving ---
  # --- CORRECTED: Combine results into a list of dictionaries ---
    print("\\nProcessing final results for this source...")
    new_rows = [] # Initialize list to store row data for the new DataFrame
    processed_indices = set() # Keep track of indices added

    # Step 1: Create a dictionary to map results back using original indices
    results_map = {}
    # Map summaries
    print(f"  Mapping {len(summary_results)} summary results...") # Debug print
    for i, result in enumerate(summary_results):
        original_index = original_indices_for_summary[i]
        if isinstance(result, Exception):
            summary_text = f"Résumé Error: {type(result).__name__}"
            print(f"  -> Summary failed for original article index {original_index}: {result}")
            logging.error(f"Async summary failed for original article index {original_index}: {result}")
        else:
            summary_text = str(result)
             # Add a basic check if summary seems empty/invalid, although API usually returns something
            if not summary_text.strip() or summary_text == "Résumé non disponible.":
                 summary_text = "Résumé Non Généré/Vide" # Use a distinct marker
        results_map[original_index] = {'summary': summary_text} # Store summary

    # Map translations (adding to the existing dictionary entry)
    print(f"  Mapping {len(translation_results)} translation results...") # Debug print
    for i, result in enumerate(translation_results):
        original_index = original_indices_for_translation[i]
        if original_index in results_map: # Check if summary existed
           if isinstance(result, Exception):
               translation_text = f"Traduction Error: {type(result).__name__}"
               print(f"  -> Translation failed for original article index {original_index}: {result}")
               logging.error(f"Async translation failed for original article index {original_index}: {result}")
           else:
               translation_text = str(result)
                # Add a basic check for empty/invalid translations
               if not translation_text.strip() or "non disponible" in translation_text.lower():
                   translation_text = "Traduction Non Générée/Vide" # Use a distinct marker
           results_map[original_index]['translation'] = translation_text # Add translation to map
        else:
            logging.warning(f"Translation result found for index {original_index} which had no corresponding summary task/result.")
            # If somehow a translation exists without a summary entry in the map
            if original_index not in results_map: results_map[original_index] = {}
            results_map[original_index]['translation'] = "Traduction sans résumé correspondant"

    # Step 2: Create the final rows using the original political_articles_data and the mapped results
    print(f"Aggregating data for {len(political_articles_data)} identified political articles...")
    for article_data in political_articles_data:
        idx = article_data["index"]
        if idx in results_map: # Ensure we have processed data for this index
            # Safely get summary and translation, providing defaults if keys are missing
            summary_text = results_map[idx].get('summary', "Résumé Manquant (Erreur Map)")
            translation_text = results_map[idx].get('translation', "Traduction Manquante (Erreur Map)")

            new_row = {
                "date": datetime.datetime.now().strftime("%Y-%m-%d %H:%M"), #  datetime.datetime.now()
                # Limit article text stored in CSV for size
                "article": article_data["text"][:3000],
                "summary": summary_text,
                "arabic_translation": translation_text,
                # Use the topic assigned during identification phase
                "topic": article_data["topic"]
            }
            new_rows.append(new_row)
            processed_indices.add(idx)
        else:
             # This indicates an article was marked political but summary/translation failed very early or mapping issue
             logging.warning(f"No summary/translation results found in map for identified political article index {idx}")
             # Optionally add a placeholder row for traceability if needed
             # new_rows.append({
             #    "date": datetime.now().strftime("%Y-%m-%d %H:%M"),
             #    "article": article_data["text"][:3000],
             #    "summary": "Erreur interne (manque résultat map)",
             #    "arabic_translation": "Erreur interne (manque résultat map)",
             #    "topic": article_data["topic"]
             # })

    print(f"Successfully collected {len(new_rows)} rows from processed articles.")
    # --- END OF CORRECTED DATA AGGREGATION ---

    # --- Previously Corrected Final Section (KEEP AS IS) ---
    # Convert the collected new rows for THIS RUN into a DataFrame.
    if new_rows:
         processed_df = pd.DataFrame(new_rows)
    else:
         # Create an empty DataFrame with correct columns if no rows were added
         # Ensure column names match EXACTLY those expected by Cell 14's optional update block if you use it.
         processed_df = pd.DataFrame(columns=['date', 'article', 'summary', 'arabic_translation', 'topic'])

    print(f"Asynchronous article processing finished for this source/file. Returning {len(processed_df)} new rows.")
    # Return ONLY the DataFrame containing the newly processed articles from this specific run
    return processed_df
    # --- End of Previously Corrected Final Section ---

In [ ]:
# Uploading one or multiple PDFs manually
from google.colab import files

uploaded_files = files.upload()

Saving Le Parisien du Mercredi 09 Avril 2025.pdf to Le Parisien du Mercredi 09 Avril 2025.pdf


In [ ]:
# CELL BEFORE 14: CapturING Pasted Text
print("Paste your article text below. Type 'END' on a new line when finished:")
pasted_lines = []
while True:
    try:
        line = input()
        if line == "END":
            break
        pasted_lines.append(line)
    except EOFError:
        break
pasted_text = "\\n".join(pasted_lines)

if pasted_text.strip():
  print("\\nPasted text captured successfully!")
else:
  print("\\nNo text pasted.")
  pasted_text = ""

Paste your article text below. Type 'END' on a new line when finished:
Philippe-Attal, la bataille qui vient dans le camp présidentiel  Tout en se ménageant, les deux ex-premiers ministres tiennent à se différencier, sans primaire, pour que l’un d’eux s’impose comme candidat unique du « bloc central ».   10 min • Louis Hausalter Loris Boichot Tristan Quinault-Maupoil É douard Philippe et Gabriel Attal se sont donné rendez-vous chez un spécialiste des rouleaux de printemps et des raviolis de crevettes. Ce 19 février, les deux anciens premiers ministres se retrouvent autour de la table de Lily Wang, un chic restaurant asiatique du 7e arrondissement de Paris. Le patron du parti présidentiel, Renaissance, invite son homologue d’Horizons à son rassemblement militant du 6 avril, prévu à Saint-Denis (Seine-Saint-Denis). Là où il prévoit d’assumer un « premier pas » vers l’élection présidentielle de 2027, à son tour. Son interlocuteur, pour sa part déjà officiellement candidat, s’est lancé dan

In [ ]:
print(pasted_text[:500])

Philippe-Attal, la bataille qui vient dans le camp présidentiel  Tout en se ménageant, les deux ex-premiers ministres tiennent à se différencier, sans primaire, pour que l’un d’eux s’impose comme candidat unique du « bloc central ».   10 min • Louis Hausalter Loris Boichot Tristan Quinault-Maupoil É douard Philippe et Gabriel Attal se sont donné rendez-vous chez un spécialiste des rouleaux de printemps et des raviolis de crevettes. Ce 19 février, les deux anciens premiers ministres se retrouvent


In [ ]:
# CELL 14: Main Processing Trigger (Handles Pasted Text AND/OR Uploaded PDFs)
import datetime

# Starting the main work of finding and summarizing political articles.

# --- STEP 0: Preparing to collect results ---

# Importing asyncio to run the main async function
import asyncio
# Importing other needed libraries (ensure pandas is imported earlier, e.g., Cell 4)
import pandas as pd
import logging # Using logging for potential error details
import traceback # Using traceback for detailed error info

# Applying nest_asyncio patch for Jupyter/Colab compatibility.
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("Applied nest_asyncio patch for Jupyter compatibility.")
except ImportError:
    print("nest_asyncio not found, proceeding without it (may cause issues in some notebook environments).")
except Exception as e:
    print(f"Error applying nest_asyncio: {e}")

# Creating an empty list (like a shopping basket) for storing results.
# We will be adding results (as DataFrames) from pasted text and PDFs into this list.
all_results_list = []
print("\n--- Ready to process text or files asynchronously ---")

# --- STEP 1: Processing Pasted Text (if any exists) ---
t_local = locals()

# Checking if the 'pasted_text' variable exists (should be defined in cell before this) and contains actual text.
if 'pasted_text' in locals() and pasted_text.strip():
    # Printing a message indicating work on pasted text is starting.
    print("\\n>>> Processing pasted text <<<")

    # --- ADDing DEBUG PRINTS  ---
    print("\\n--- Debugging Variable Availability ---")
    print(f"Checking for 'political_keywords': {'political_keywords' in locals()}")
    print(f"Type of 'political_keywords': {type(political_keywords) if 'political_keywords' in locals() else 'Not Found'}")
    print(f"Checking for 'classifier': {'classifier' in locals()}")
    print(f"Type of 'classifier': {type(classifier) if 'classifier' in locals() else 'Not Found'}")
    print(f"Checking for 'async_client': {'async_client' in locals()}")
    print(f"Type of 'async_client': {type(async_client) if 'async_client' in locals() else 'Not Found'}")
    print(f"Checking for 'classification_threshold': {'classification_threshold' in locals()}")
    print(f"Type of 'classification_threshold': {type(classification_threshold) if 'classification_threshold' in locals() else 'Not Found'}")
    print(f"Checking for 'min_keyword_hits': {'min_keyword_hits' in locals()}")
    print(f"Type of 'min_keyword_hits': {type(min_keyword_hits) if 'min_keyword_hits' in locals() else 'Not Found'}")
    print("--- End Debugging ---")
    # --- END DEBUG PRINTS ---


    # Using a 'try...except' block for handling potential errors during processing.
    try:
        # Verifying all necessary variables for processing exist before calling the function.
        required_vars_paste = ['political_keywords', 'classifier', 'async_client', 'classification_threshold', 'min_keyword_hits']

        if all(v in t_local for v in required_vars_paste):
             # Calling the main asynchronous function 'async_process_articles_from_text' (defined in Cell 12).
             # Using the correct parameter names as defined in Cell 12.

             results_df_pasted = await async_process_articles_from_text(
                  full_text=pasted_text,
                  political_keywords=political_keywords,
                  async_client=async_client, # The OpenAI connection object
                  summaries_df=summaries_df,
                  csv_path=csv_path
             )
             # Checking if any political articles were actually found in the pasted text.
             if not results_df_pasted.empty:
                 # Adding the results table (DataFrame) to our list if results were found.
                 print(f"Found {len(results_df_pasted)} political articles in pasted text.")
                 all_results_list.append(results_df_pasted)
             else:
                 # Printing a message if no political articles were identified in the pasted text.
                 print("No political articles identified in pasted text.")
             # Printing a message indicating completion for pasted text.
             print(">>> Finished processing pasted text <<<\\n")
        else:
            # If required variables are missing
            missing_vars = [v for v in required_vars_paste if v not in t_local]
            print(f"❌ Error: Cannot process pasted text. Missing required variables: {missing_vars}.")

    except Exception as e:
        # Printing the error message if an error occurred during processing.
        logging.error(f"Critical error processing pasted text: {e}\n{traceback.format_exc()}")
        print(f"❌ Critical error processing pasted text: {e}")
        traceback.print_exc() # Printing detailed error traceback

# --- STEP 2: Processing Uploaded PDF Files (if any exist) ---

# Importing the 'os' library needed for file operations (like deleting the temp file)
import os

# Checking if the 'uploaded_files' variable exists (should be created by Cell 13) and if it contains any files.
if 'uploaded_files' in locals() and uploaded_files:
    # Printing a message indicating processing of PDF files is starting.
    print("--- Processing Uploaded Files ---")
    # Keeping track of the number of successfully processed PDFs.
    processed_files_count = 0
    # Looping through each uploaded file.
    # 'filename' is the name (e.g., "MyFile.pdf").
    # 'file_data' holds the actual content (bytes) of the uploaded file.
    for filename, file_data in uploaded_files.items():
        # Printing the name of the file currently being processed.
        print(f"\\n>>> Processing file: {filename} <<<")
        logging.info(f"Processing file: {filename}") # Logging which file is being processed

        # Defining the path where we'll temporarily save the PDF in Colab's file system.
        # It will be saved in the main '/content/' directory with its original name.
        temp_pdf_path = f"/content/{filename}"

        # Using 'try...except...finally' for safer handling.
        # The 'finally' part ensures the temporary file gets deleted even if errors happen.
        pdf_text = None # Resetting pdf_text for each new file.
        try:
            # --- Saving the uploaded bytes to a temporary file ---
            # Printing a status message.
            print(f"Saving temporary file: {temp_pdf_path}...")
            # Opening the temporary file path in 'write bytes' mode ('wb').
            with open(temp_pdf_path, "wb") as f:
                # Writing the raw byte content stored in 'file_data' into the temporary file.
                f.write(file_data)
            # Printing confirmation that the temporary file was saved.
            print("Temporary file saved.")

            # --- Extracting text from the temporarily saved PDF file ---
            # Printing a status message.
            print(f"Extracting text from {temp_pdf_path}...")
            # Calling the function from Cell 10, passing the PATH to the temporary file.
            pdf_text = extract_text_from_pdf(temp_pdf_path)
            # Printing whether text extraction succeeded or failed based on if 'pdf_text' got content.
            print(f"Text extraction {'succeeded' if pdf_text else 'failed'}.")

            # --- Processing the extracted text (if successful) ---
            # Checking if the 'pdf_text' variable now contains extracted text.
            if pdf_text:
                 # Verifying all necessary variables (keywords, model, client, etc.) exist before calling the main processing function.
                 # Using the CORRECT variable names ('classifier', 'async_client').
                 required_vars_pdf = ['political_keywords', 'classifier', 'async_client', 'classification_threshold', 'min_keyword_hits']
                 if all(v in t_local for v in required_vars_pdf):
                     # Calling the main processing function 'async_process_articles_from_text' from Cell 12.
                     # Passing the extracted 'pdf_text' and using the CORRECT variable names for arguments.
                     results_df_pdf = await async_process_articles_from_text(
                        full_text=pasted_text,
                        political_keywords=political_keywords,
                        async_client=async_client, # The OpenAI connection object
                        summaries_df=summaries_df,
                        csv_path=csv_path
                     )
                     # Checking if the processing function found any political articles in this PDF.
                     if not results_df_pdf.empty:
                          # Adding the results table (DataFrame) for this PDF to our main list 'all_results_list'.
                          print(f"Found {len(results_df_pdf)} political articles in {filename}.")
                          all_results_list.append(results_df_pdf)
                          processed_files_count += 1 # Counting this file as successfully processed.
                     else:
                         # Printing a message if no political articles were found in this PDF after filtering.
                         print(f"No political articles identified in {filename}.")
                 else:
                    # Printing an error if some required setup variables were missing.
                    missing_vars = [v for v in required_vars_pdf if v not in t_local]
                    print(f"❌ Error: Cannot process file {filename}. Missing required variables: {missing_vars}.")
            else:
                # Printing a warning if text could not be extracted from the PDF.
                logging.warning(f"Skipping file {filename} due to PDF extraction error.")
                print(f"⚠️ Skipping file {filename} due to PDF extraction error.")

        except Exception as e:
            # Catching any unexpected error during the processing of this specific file.
            logging.error(f"Critical error processing file {filename}: {e}\\n{traceback.format_exc()}")
            print(f"❌ Critical error processing file {filename}: {e}")
            traceback.print_exc() # Printing the detailed error information.
        finally:
            # --- Cleaning up the temporary file ---
            # This 'finally' block always runs, whether errors happened or not.
            # Checking if the temporary PDF file we created still exists.
            if os.path.exists(temp_pdf_path):
                 try:
                     # Trying to delete the temporary file.
                     os.remove(temp_pdf_path)
                     print(f"Removed temporary file: {temp_pdf_path}")
                 except Exception as remove_e:
                     # Printing a warning if the temporary file couldn't be deleted.
                     print(f"⚠️ Error removing temporary file {temp_pdf_path}: {remove_e}")

        # Printing a message indicating completion for the current file.
        print(f">>> Finished processing for file: {filename} <<<")

    # Printing a summary after trying to process all uploaded files.
    print(f"\\n--- Processed {processed_files_count} uploaded file(s) ---")

else:
    # Printing a message if no files were uploaded in Cell 13.
    print("--- No files uploaded to process ---")



# --- NEW STEP: Post-Filtering Noise ---
print("\\n--- Applying Post-Filtering Rules ---")
cleaned_results_list = []
noise_keywords = [
    'loto', 'météo', 'hippique', 'courses', 'turf', 'tiercé', 'quarté', 'quinté', 'nécrologie', 'avis de décès',
    'psg', 'aston villa', 'football', 'ligue des champions', 'match', 'gardien', 'buts', 'joueurs', 'championnat'
    # We can Add any other sports terms you noticed in the irrelevant summaries
]
# Loop through each DataFrame in the results list
for df_result in all_results_list:
    if not df_result.empty and 'summary' in df_result.columns:
        # Keep rows where the summary does NOT contain any noise keyword (case-insensitive)
        df_cleaned = df_result[~df_result['summary'].str.contains('|'.join(noise_keywords), case=False, na=False)]
        if not df_cleaned.empty:
            cleaned_results_list.append(df_cleaned)
            print(f"  Kept {len(df_cleaned)} articles after filtering {len(df_result)} initial results.")
        else:
            print(f"  Filtered out all {len(df_result)} articles based on noise keywords.")
    else:
         # Keeping the DataFrame as is if it's empty or doesn't have a summary column
         cleaned_results_list.append(df_result)

# Replacing the original list with the cleaned one
all_results_list = cleaned_results_list
print("--- Finished Post-Filtering ---")
# --- END OF NEW STEP ---

# --- STEP 3: Combining and Saving All Results ---
# (The existing code below will now use the 'cleaned_results_list')
print("\n--- Combining and Saving Results for This Run ---")

# Checking if the list (containing DataFrames from this run after post-filtering) is not empty
if all_results_list:
    # Combining all individual results DataFrames from THIS RUN into one DataFrame.
    # This aggregates results from pasted text AND pdf files processed in this execution.
    current_run_df = pd.concat(all_results_list, ignore_index=True)

    num_processed_articles = len(current_run_df) # Count articles processed and kept FROM THIS RUN
    print(f"\nProcessed and kept {num_processed_articles} articles in this run (after post-filtering).")

    # Proceed only if there are articles to save from this run
    if num_processed_articles > 0:
        # --- Defining the save path for the final CSV with Timestamp ---
        # Get the current date and time
        now = datetime.datetime.now()
        # Format it nicely for a filename (e.g., YYYYMMDD_HHMMSS)
        timestamp = now.strftime("%Y%m%d_%H%M%S")
        # Define the base path and ensure it exists
        import os
        base_output_path = "/content/drive/MyDrive/RadioBrief/"
        os.makedirs(base_output_path, exist_ok=True)
        # Create the new filename including the timestamp
        final_csv_path = os.path.join(base_output_path, f"radio_summaries_{timestamp}.csv")
        print(f"Results for this run will be saved to: {final_csv_path}")

        # --- Saving the results FROM THIS RUN to the timestamped CSV file ---
        current_run_df.to_csv(final_csv_path, index=False, encoding='utf-8-sig')
        print(f"\nSaved {num_processed_articles} political articles results from THIS RUN to CSV: {final_csv_path}")

        # --- (Optional) Update the main cumulative file ---
        # This block loads the old main file, adds the new unique articles from this run,
        # removes duplicates (keeping the latest article text), and saves back.
        # we can comment out or remove this block if we ONLY want uniquely timestamped files per run.
        try:
            main_csv_file_path = "/content/drive/MyDrive/RadioBrief/radio_summaries.csv" # Adjust if needed
            if os.path.exists(main_csv_file_path):
                summaries_df = pd.read_csv(main_csv_file_path)
                # Ensure consistent columns before concatenating
                # Make sure both dataframes have the same columns in the same order
                cols = current_run_df.columns.tolist()
                summaries_df = summaries_df.reindex(columns=cols)
                combined_df = pd.concat([summaries_df, current_run_df], ignore_index=True) # Combine old with current run
            else:
                combined_df = current_run_df # If main file doesn't exist, it's just the current run

            # Remove duplicates based on 'article' column, keeping the latest entry
            combined_df.drop_duplicates(subset=['article'], keep='last', inplace=True)
            combined_df.to_csv(main_csv_file_path, index=False, encoding='utf-8-sig')
            print(f"Updated main cumulative file: {main_csv_file_path} with {len(combined_df)} unique articles total.")
        except Exception as e:
            print(f"Error updating main cumulative file: {e}")
        # --- End Optional Update ---

    else:
         print("\nNo articles remained after post-filtering in this run to save.")

else:
    # Printing a message if no results were generated from any source during this run.
    print("\\nNo results generated in this run to save.")

# --- STEP 4: Finishing Up --- (Keep this part as is)
print("\\n--- Processing Complete ---")
# --- STEP 4: Finishing Up ---
print("\\n--- Processing Complete ---")

Applied nest_asyncio patch for Jupyter compatibility.

--- Ready to process text or files asynchronously ---
\n>>> Processing pasted text <<<
\n--- Debugging Variable Availability ---
Checking for 'political_keywords': True
Type of 'political_keywords': <class 'list'>
Checking for 'classifier': True
Type of 'classifier': <class 'transformers.pipelines.zero_shot_classification.ZeroShotClassificationPipeline'>
Checking for 'async_client': True
Type of 'async_client': <class 'openai.AsyncOpenAI'>
Checking for 'classification_threshold': True
Type of 'classification_threshold': <class 'float'>
Checking for 'min_keyword_hits': True
Type of 'min_keyword_hits': <class 'int'>
--- End Debugging ---
\nStarting asynchronous article processing...
--- Debugging smart_split_articles ---
Cleaned text (first 500 chars):
Philippe-Attal, la bataille qui vient dans le camp présidentiel  Tout en se ménageant, les deux ex-premiers ministres tiennent à se différencier, sans primaire, pour que l’un d’eux s’i

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


  (Raw classification: Label='Relations Internationales', Score=0.6720)
topic_result
{'sequence': 'est un pays de médecine et de soins qui a su défendre son originalité dans le domaine de l’éthique. À partir d’une réflexion approfondie et sur la base d’expériences internationales, notre pays peut proposer une troisième voie, une voie originale française, inventer des maisons d’accompagnement, imaginer de nouvelles approches des soins palliatifs, voire envisager la prise en compte de la spiritualité en fin de vie. Le moment n’est-il pas venu de nous interroger sur la place de la liberté et de la vulnérabi', 'labels': ['Relations Internationales', 'Société', 'Gouvernement', 'Économie', 'Élections', 'Diplomatie', 'Politique'], 'scores': [0.6719722151756287, 0.08599963039159775, 0.07951637357473373, 0.05085674300789833, 0.0415329709649086, 0.04027858003973961, 0.029843363910913467]}
topic_result
  -> Article 8 identified as 'Relations Internationales' by classifier.
  (Classifying text sta

In [ ]:
summaries_df.head()

,date,article,summary,arabic_translation,topic
0,2025-04-15 17:52,PUBLICITÉParis •Mercredi 9 avril 2025 • N° 250...,Un incendie sans danger pour la santé a eu lie...,وقع حريق غير ضار للصحة في مركز جمع وفرز النفاي...,Football
1,2025-04-17 05:04,UNE MARCHE\nAPRÈS L’AUTRE\nUn mois après son e...,"Un mois après sa victoire contre Liverpool, le...",بعد شهر من فوزه على ليفربول، يستعد باريس سان-ج...,Vous n'avez pas fourni de texte à lire. Veuill...
2,2025-04-17 05:04,"sélectionneur\nEmery (53 ans, ESP)\nen pos te ...","L'entraîneur du Paris Saint-Germain, Luis Enri...",مدرب باريس سانت جيرمان، لويس إنريكي، أعلن أن غ...,Vous n'avez pas fourni de texte à lire. Veuill...
3,2025-04-17 05:05,"Dembélé, un soir pour savoir\nLe Parisien n’a ...","L'attaquant français Ousmane Dembélé, qui joue...",المهاجم الفرنسي أوسمان ديمبيلي، الذي يلعب حالي...,Vous n'avez pas fourni de texte à lire. Veuill...
4,2025-04-17 05:05,football Ligue des champions quarts de finale ...,"Le gardien de but argentin d'Aston Villa, Emil...",حارس مرمى أستون فيلا الأرجنتيني، إميليانو مارت...,"Comme il n'y a pas de texte fourni, je ne peux..."


In [ ]:
summaries_df.tail(10)

,date,article,summary,arabic_translation,topic
143,2025-04-22 10:49,"», une unité rassemblant des volontaires du mo...",Des volontaires du monde entier rejoignent une...,ينضم متطوعون من جميع أنحاء العالم إلى وحدة قوم...,Diplomatie
144,2025-04-22 10:49,Giorgia Meloni gagne la bienveillance de Donal...,Giorgia Meloni a reçu des éloges de la part de...,تلقت جورجيا ميلوني إشادة من دونالد ترامب خلال ...,Gouvernement
145,2025-04-22 10:49,des États-Unis. Mais Vance joue aussi le rôle...,Le vice-président Vance joue un rôle clé dans ...,يقوم نائب الرئيس فانس بدور رئيسي في العلاقات ا...,Politique
146,2025-04-22 10:49,. Elle est frappée par l’atmosphère de violenc...,Les jeunes cherchent un sens à leur vie dans u...,يبحث الشباب عن معنى لحياتهم في سياق العنف، وفق...,Gouvernement
147,2025-04-22 10:49,. Comment échapper à cette importation du conf...,Un débat sur la place de la religion et de la ...,يثار جدل حول دور الدين والإيمان في مجتمعاتنا، ...,Société
148,2025-04-22 10:49,"chinoise, dépendante de ses exportations. J. G...",L'État français investit 400 millions d'euros ...,تستثمر الحكومة الفرنسية 400 مليون يورو في تخفي...,Économie
149,2025-04-22 10:49,"2030, un plan d’investissement de 54 milliards...",Le gouvernement a lancé un plan d'investisseme...,أطلقت الحكومة خطة استثمارية بقيمة 54 مليار دول...,Gouvernement
150,2025-04-22 10:49,", nourrit aussi les inquiétudes, tout comme l’...",L'instabilité politique en France et la dette ...,تثير عدم الاستقرار السياسي في فرنسا وديونها ال...,Politique
151,2025-04-22 10:49,"Inquiets de Trump, des riches Américains dépla...",Des riches Américains déplacent leur fortune e...,يقوم الأثرياء الأمريكيون بتحويل ثرواتهم إلى سو...,Politique
152,2025-04-22 10:49,. « Les navires et le transport maritime sont ...,"Le représentant américain au Commerce, Jamieso...",أكد الممثل الأمريكي للتجارة، جيميسون جرير، على...,Gouvernement



# 📘 RadioBrief – Final Project Report: Automated Political Press Review

## 1.  Project Goal & Pipeline Overview

The goal of the **RadioBrief** project was to automate the political press review process. The system is designed to extract, classify, summarize, and translate political news from French newspaper PDFs. The output is saved to a structured CSV format for editorial use.

**Pipeline Components:**
- PDF or pasted text ingestion
- Cleaning & preprocessing
- Article segmentation via `smart_split_articles`
- Topic classification (CamemBERT-based model)
- Political filtering (score + keyword density)
- Summary generation via OpenAI API
- Arabic translation via OpenAI API
- Asynchronous processing (for performance)
- Final export to timestamped CSV

---

## 2.  Iterative Development and Key Milestones

###  Baseline Setup
- Keyword list: ~85
- Thresholds: score ≥ 0.4, hits ≥ 2
- Result: Only 6 articles captured (too strict).

###  Iteration 1: Keyword Expansion
- Expanded list to ~177 (Gaza, Égypte, migration, etc.)
- Result: Still 6 articles. Splitting or scoring likely bottlenecks.

###  Iteration 2: Threshold Relaxation
- score ≥ 0.3, hits ≥ 1
- Result: Captured 15 articles (Macron, Sciences Po).
- Introduced noise (obituaries, lottery, etc.)

###  Iteration 3: Re-tightening Thresholds
- score ≥ 0.35, hits ≥ 1
- Result: 8 articles, less noise but Macron diplomacy excluded.

###  Iteration 4: Adjusting Article Splitting
- Removed ALL CAPS split
- Result: Still 15 articles, Macron/Gaza article still missing.

###  Iteration 5: Filter Refinement
- Removed redundant `len < 300` filter
- min_keyword_hits = 3
- Result: Captured key articles and reduced noise.

###  Iteration 6: Final Tuning & Bug Fixes
- Fixed `datetime.now()` bug
- Introduced post-filtering for known noise terms
- Result: 11 solid articles from PDF, 25 from pasted input.
- Final filter: score ≥ 0.3, hits ≥ 3

---

## 3.  Final Results & Analysis

###  Strengths
- End-to-end functional system
- Handles both PDF & pasted input
- Asynchronous processing boosts efficiency
- Accurately captures relevant political articles
- Excludes noise effectively (lottery, sports)
- Timestamped CSVs for clean output tracking

###  Weaknesses
- Chunking still based on heuristics, prone to edge cases
- Zero-shot classifier lacks perfect precision
- Logging print statements could be improved

---

## 4.  Future Work & Expansion

###  Splitting Improvements
- Refine section_keywords
- Test NLP-based segmentation (sentence similarity, coherence)

###  Global Summary Generation ("Résumé Global")
- Concatenate summaries by topic
- Generate group-level summaries ("nashra" format)

###  Streamlit App (Pending)
- File upload, result filters, result previews
- Future phase: article group synthesis + account system

###  Fine-Tuning (Planned)
- Classifier: Train CamemBERT on political news datasets
- Summarizer: Fine-tune to create stylized "radio-ready" summaries

###  Evaluation Framework
- Label articles manually
- Measure precision, recall, F1 for threshold tuning

---

## 5.  Debugging Session Report (Copie_de_Rafael.ipynb – April 22, 2025)

###  Goal:
Fix bugs affecting article counts, CSV saving behavior, and topic classification reliability.

###  Fixes Applied:

####  Cell 14: Saving Logic
- Problem: Results were cumulatively saved, inflating outputs
- Fix: Now saves only **current_run_df**, not appended results
- Outcome: Accurate counts (29 articles) and correct CSV saving

####  Cell 9: Topic Classification Errors
- Problem: Classification received empty/invalid texts
- Fix: Input validation added before model call
- Outcome: Avoided runtime errors, stable classification

####  Cell 12: Async Return & Data Assembly
- Problem: Function returned entire dataset instead of new results
- Fix: Now returns only `processed_df` with new summaries
- Outcome: Accurate row count and modular returns

####  Cell 12: Data Aggregation Fix
- Problem: Async summaries weren’t mapped properly to results
- Fix: Corrected `new_rows` logic, properly indexes summary outputs
- Outcome: Final DataFrame correctly reflects API responses

####  Cell 12: datetime.now() Bug
- Problem: AttributeError due to incorrect datetime usage
- Fix: Changed to `datetime.datetime.now()`
- Outcome: Timestamps generated correctly

---

##  Final Output Confirmation
- ✔️ Total 36 articles (11 PDF, 25 paste)
- ✔️ Macron/Gaza article captured
- ✔️ Noise removed
- ✔️ No classification or runtime errors

---

 **Conclusion:**  
The RadioBrief pipeline delivers on its promise—an efficient, multilingual political press review assistant with room to grow through app and model fine-tuning.


## copied text from Le Figaro du Mercredi 09 Avril 2025
Philippe-Attal, la bataille qui vient dans le camp présidentiel  Tout en se ménageant, les deux ex-premiers ministres tiennent à se différencier, sans primaire, pour que l’un d’eux s’impose comme candidat unique du « bloc central ».   10 min • Louis Hausalter Loris Boichot Tristan Quinault-Maupoil É douard Philippe et Gabriel Attal se sont donné rendez-vous chez un spécialiste des rouleaux de printemps et des raviolis de crevettes. Ce 19 février, les deux anciens premiers ministres se retrouvent autour de la table de Lily Wang, un chic restaurant asiatique du 7e arrondissement de Paris. Le patron du parti présidentiel, Renaissance, invite son homologue d’Horizons à son rassemblement militant du 6 avril, prévu à Saint-Denis (Seine-Saint-Denis). Là où il prévoit d’assumer un « premier pas » vers l’élection présidentielle de 2027, à son tour. Son interlocuteur, pour sa part déjà officiellement candidat, s’est lancé dans une série de meetings régionaux.  Les temps sont à la préparation de l’après-Macron, sous la surveillance de figures du « bloc central » méfiantes devant ce duel d’ambitieux anciens premiers ministres. « Avant de se demander qui porte un projet, il faudrait qu’il y ait un projet, pointe la ministre de l’Éducation nationale, Élisabeth Borne, face à ce qu’elle appelle la « bande des garçons ». Il faudra envoyer des messages de ce qu’on veut faire, et non pas juste dire : “Je suis là.” » C’est maintenant en dehors du pouvoir exécutif, tournés vers l’avenir, que s’organisent l’ex-Républicain (LR) et l’ancien socialiste, tous deux devenus personnalités politiques préférées des Français à leur sortie du gouvernement. Pour la présidentielle, « ce sont les deux plus attendus, les plus crédibles dans le bloc central », remarque un cadre de Renaissance. Et d’ajouter, beau joueur : « Quoi qu’il arrive, le futur du pays se fera avec Édouard Philippe et Gabriel Attal. »  Entre le maire du Havre (Seine-Maritime), 54 ans, et son rival, de dix-huit ans son cadet, il y aurait de la cordialité, « sans tensions », selon l’entourage du premier, ni « guerre larvée » ou « animosité », selon les proches du second. Tous deux membres du même gouvernement de 2018 à 2020, ils partagent des orientations semblables - proeuropéennes, libérales en économie et fermes en matière régalienne. Mais les divergences ne manquent pas. La polémique récente sur le port du voile lors des compétitions sportives en a apporté l’illustration. Le président d’Horizons a exprimé ses réticences devant la proposition de loi de LR visant à l’interdire ; Gabriel Attal en a profité pour s’y engouffrer et marquer sa différence. Une manière de rassurer sa droite, alors qu’il cherche à apparaître comme le garant du « dépassement » des clivages gauche-droite, d’une forme de « en même temps », davantage remis en cause par Édouard Philippe, autoproclamé « homme de droite ». À l’égard de François Bayrou, le trentenaire a fait le choix de la bienveillance, alors qu’Édouard Philippe répète qu’aucune réforme majeure ne pourra voir le jour d’ici la prochaine présidentielle.  « Mauvaise décision »    Chacun à sa manière, tous deux entendent marquer leur différence vis-à-vis de la méthode du président de la République. Nommé premier ministre plus récemment, aujourd’hui à la tête du parti du chef de l’État, Gabriel Attal sait que l’exercice est plus difficile pour lui. Il s’est contenté d’appeler ses troupes à « se réinventer sans se renier », lors de son meeting de Saint-Denis. Édouard Philippe hésite moins à se distinguer d’Emmanuel Macron, quand il fustige sa « mauvaise décision » de dissoudre l’Assemblée nationale, rendue « ingouvernable ». Et, en affirmant que sa précédente erreur a été de remplacer Élisabeth Borne dès le mois de janvier 2024, se privant ainsi d’une réponse politique après les élections européennes, il prend aussi ses distances avec la nomination de Gabriel Attal, jugée trop précoce. Point de départ d’un affrontement que le Normand anticipait lui-même dans Le Parisien : « Tout le monde va vouloir jouer le match Attal-Philippe. C’est aussi attendu que sans intérêt. »  Leurs stratégies diffèrent aussi vis-à-vis du Rassemblement national (RN). Gabriel Attal se montre le plus véhément. Des accusations de collusion avec le « trumpisme », des phrases courtes et percutantes… « Tu voles, tu paies », a-t-il lancé à Saint-Denis le 6 avril, à l’adresse de Marine Le Pen, condamnée notamment à l’inéligibilité immédiate pour détournement de fonds publics - elle a fait appel. N’a-t-il pas entendu Édouard Philippe, quelques heures plus tôt sur la même scène, appeler à privilégier « un projet clair » plutôt que « l’attaque de son adversaire » ? Les philippistes, peu adeptes de « phrases acides », se méfient des « punchlines » des attalistes. « Je ne sais pas si ça fait des bons présidents de la République », glisse Laurent Marcangeli, ministre (Horizons) de la Fonction publique.  Le trentenaire est certain d’avoir marqué les esprits, lors de la campagne législative, en entretenant un duel frontal avec Jordan Bardella. Au point de ne pas reprendre à son compte la ligne « ni RN, ni La France insoumise (LFI) » qu’Édouard Philippe a défendue dans l’entre-deux tours. Il lui a préféré l’appel à un large front républicain anti-RN, incluant les Insoumis, quitte à mécontenter une partie de son camp. « L’appel au vote LFI pour faire barrage au RN est une cassure avec les électeurs de droite », souligne Aurore Bergé, ministre déléguée chargée de l’Égalité femmes-hommes.  À l’Élysée, on observe le duel sans prendre parti. Moins par souci de neutralité que parce qu’aucun des deux protagonistes n’a les faveurs d’Emmanuel Macron. Ni Gabriel Attal, ni Édouard Philippe : tel est l’état d’esprit du chef de l’État, qui garde de mauvais souvenirs de chacun de ces ex-premiers ministres, coupables de lui avoir fait de l’ombre. Ce casting ne plaît pas non plus aux historiques de la macronie.  Le 31 mars, Yaël Braun-Pivet a convié à dîner à l’hôtel de Lassay plusieurs anciens lieutenants du président. Autour de la table de la présidente de l’Assemblée : les ex-ministres Julien Denormandie, Stanislas Guerini et Olivier Dussopt, l’ancien conseiller élyséen Philippe Grangeon et le haut-commissaire au Plan, Clément Beaune. La discussion dérive forcément sur 2027, alors qu’Emmanuel Macron ne peut pas se représenter, que Marine Le Pen sera peut-être empêchée, et que Gabriel Attal s’apprête à tenir son meeting quelques jours plus tard, où résonneront les cris « Attal président ! ».  Un slogan que ne goûtent pas tellement les convives de cette soirée. « Ils ont toujours l’esprit de 2017 en commun. Ils constatent ce qui a foiré, mais ils veulent absolument poursuivre dans cette ligne, alors qu’ils sont dans un parti en cours de captation par Attal », témoigne-t-on dans l’entourage de l’un des participants.  Derrière cette volonté affichée de faire vivre le « dépassement », il y a celle, moins avouable, d’empêcher Gabriel Attal de se poser en successeur naturel d’Emmanuel Macron. Tout comme de concurrencer Édouard Philippe, dont cette petite bande qui a plutôt le cœur à gauche ne partage pas les options idéologiques. « Le président n’a pas de successeur, tranche l’un de ses proches. Et quand il n’y a pas de dauphin s’épanouissent les requins. Nous allons vers un duel fratricide entre Philippe et Attal. »  « La clé du succès, c’est l’unité »   Le jeu reste ouvert, sur fond d’éclatement au centre, à gauche et - pourquoi pas - dans le camp nationaliste, en cas d’empêchement de Marine Le Pen. « Si le seuil de qualification au second tour s’abaisse, ça peut ouvrir toutes les velléités, poursuit un autre proche du chef de l’État. Ce qui est assez inédit dans une présidentielle, c’est le nombre de personnalités qui ont les moyens financiers organisationnels d’y aller. Philippe, Attal, Bayrou ou Retailleau ont l’argent et les structures pour tous être candidats. »  Ce qui ne va pas tellement dans le sens du mot d’ordre seriné dans l’entourage d’Emmanuel Macron : « La clé du succès, c’est l’unité ». Au contraire, « le bloc central est voué à l’explosion », prédit un interlocuteur régulier du président, qui invite à regarder vers la gauche : « À force de répéter que la France est de droite, Philippe et Attal finissent par en être persuadés. Mais un espace de centre gauche s’est ouvert ». À l’Élysée, on observe de près les intentions de Bernard Cazeneuve et Raphaël Glucksmann, compatibles avec une gauche macroniste en quête d’un candidat en 2027.  Alors qu’Édouard Philippe plaide pour un rassemblement « de la droite conservatrice à la social-démocratie », les proches de Gabriel Attal jugent leur champion le mieux placé pour s’adresser à la gauche modérée. « Quand Attal dit “je suis travailliste”, c’est malin, avance le député (Renaissance) Antoine Armand. Ne perdons pas l’électorat qui nous permet de tenir du centre droit au centre gauche. »  Point de convergence entre les deux anciens premiers ministres : le refus de la primaire de départage proposée par Gérald Darmanin. Édouard Philippe est persuadé que l’évidence de sa pole position dans les sondages s’imposera à tous ceux qui chercheraient à le concurrencer. « Il est aujourd’hui le seul candidat capable de battre Marine Le Pen avec sérieux et crédibilité », estime la porte-parole d’Horizons, Naïma Moutchou, qui met en garde contre le scénario « suicidaire » d’une division : « Ça reviendrait à offrir les clés du pays au RN. Quand l’heure des choix viendra, chacun se remettra autour de la table. »  Stratégie d’implantation locale   « Si certains parlent d’une primaire, c’est bien qu’il y a match », observe la garde rapprochée de Gabriel Attal. « Aucune idée de comment ça va se décanter, c’est trop tôt pour le dire », glisse Maxime Cordier, son plus proche conseiller. Chez Horizons, élus et militants sont persuadés que leur stratégie d’implantation locale finira par payer, face à un parti macroniste qu’ils jugent en mal d’enracinement. Beaucoup saluent le choix d’Édouard Philippe de ne pas s’être présenté aux élections législatives pour conserver sa mairie, alors que Gabriel Attal a mené la campagne du camp présidentiel et pris la tête des députés Renaissance. « Je préfère amplement faire au Havre et réfléchir en France, plutôt que de parler à Paris », assumait Édouard Philippe auprès du Figaro, en novembre dernier.  Ce dernier tarde à présenter son programme, prévu après les élections municipales de mars 2026, alors que Gabriel Attal organisera plusieurs conventions thématiques à partir du 5 mai. « Il y a une attente pour qu’il éclaircisse ses dorsales idéologiques. On va le faire, mais pas en miroir vis-à-vis de qui que ce soit », relativise-t-on à Renaissance. « On ne se déterminera pas par rapport à Attal », assurent de leur côté les dirigeants d’Horizons. Comme pour retarder le choc attendu. L.H. ET L.B. ET T.Q.-M. Définir l’après-macronisme, le véritable enjeu   3 min • Guillaume Tabard C’est un des traits d’esprit dont il avait le secret. « Ce qui est à redouter, ce n’est pas le vide politique, c’est le trop-plein », disait le général de Gaulle, interrogé sur sa succession. Emmanuel Macron n’est pas de Gaulle… Mais après deux mandats présidentiels, la question de l’avenir de son camp politique est posée. Pour relever le gant, les prétendants ne manquent pas. Parmi ceux-ci, Édouard Philippe est le seul à s’être déclaré officiellement candidat mais Gabriel Attal n’est pas celui qui y pense le moins. L’après-macronisme ne se limite cependant pas à une bataille de leadership qu’une primaire, les sondages ou un conclave suffirait à arbitrer.  Les deux hommes ont en commun d’avoir été premiers ministres d’Emmanuel Macron, l’un en premier, l’autre en dernier. Pour autant, l’enjeu de 2027 n’est pas de savoir lequel est le mieux à même de reprendre l’héri­tage. D’abord parce que l’occupant actuel de l’Élysée n’a pas la moindre envie de tester en faveur de l’un ou de l’autre. Aucun n’est le favori de Macron, comme Laurent Fabius l’avait été de Mitterrand et Alain Juppé de Chirac, même si, pour toutes sortes de raisons, ce statut ne leur fut d’aucun secours. Mais surtout parce que ni Philippe ni Attal n’ont l’intention de se poser en héritiers. Successeur, oui, continuateur, non.  Nous ne sommes pas non plus dans le cas de figure d’un Nicolas Sarkozy s’étant construit en opposition avec Chirac au point de réussir à prendre sa suite, avec le même périmètre politique que lui, en ayant fait campagne sur le thème de la « rupture ». Aujourd’hui, la question n’est pas person­nelle, même si les patrons de Renaissance et d’Horizons s’estiment libérés de tout engagement envers le chef de l’État pour avoir été remerciés de ­Matignon trop tôt à leur goût.  La question de fond est cependant de savoir s’il s’agit de prolonger le macronisme ou d’inventer quelque chose de nouveau. De pérenniser l’ambition du « dépassement » politique ou de revenir au clivage gauche-droite. Tout en sachant qu’en 2027 la confrontation devrait se faire, pour la troisième fois consécutive, face au représentant du Rassemblement national. Et qu’est-ce qui dominera dans deux ans : la volonté de faire barrage à une victoire du RN qui n’aura jamais été aussi envisageable ou l’envie de tourner à tout prix la page d’un double quinquennat qui, par-delà les crises imprévues (Covid, Ukraine, Trump…), n’aura pas apporté la « révolution » promise ? Ces questions se poseront à Édouard Philippe et Gabriel Attal mais aussi à tous ceux qui auront gouverné durant cette décennie.  L’architecture politique à construire est une autre inconnue. Édouard ­Philippe prétend rassembler « de la social-démocratie à la droite conserva­trice », ce qui supposerait d’être reconnu en candidat commun de tout ce spectre. Gabriel Attal refuse de considérer naturel ou durable de gouverner avec LR, ce qui supposerait de pouvoir soutenir la concurrence d’un candidat de droite. Mais tout cela ne dépend pas que d’eux. Et puisque dans tous les partis du « socle commun », envisager une procédure de désignation validée par tous est un tabou, le flou sur les contours de l’après-Macron risque de persister longtemps. G. T. Jean Leonetti : « Avec la nouvelle loi sur la fin de vie, tous les verrous sauteront tôt ou tard »  Vingt ans après sa loi sur la fin de vie, le médecin et élu LR pointe les non-dits du projet gouvernemental et les dangers d’une transgression fondamentale.   7 min • Emmanuel Galiero J ean Leonetti, maire LR d’Antibes, a donné son nom à la loi du 22 avril 2005 relative aux droits des malades et à la fin de vie ainsi qu’à la loi du 2 février 2016 (Claeys-Leonetti) relative aux droits des patients en fin de vie.  LE FIGARO. - LE FIGARO. - La loi relative aux droits des malades et à la fin de vie, qui porte votre nom, aura vingt ans le 22 avril. Dans quel contexte fut-elle votée ?  - JEAN LEONETTI. - La loi de 2005 fait suite au drame très médiatisé de Vincent Humbert qui relance le débat sur la légalisation de l’euthanasie. Les débats et les auditions durent plus d’un an et progressivement un doute collectif se substitue à des certitudes individuelles, ce qui permet la convergence de points de vue initialement opposés. Entre le statu quo et la légalisation de l’euthanasie, une troisième voie originale finit par s’imposer. Il s’agit du triple engagement du « non-abandon », de la « non-souffrance » et du rejet de « l’acharnement thérapeutique ». Le texte adopté à l’unanimité apparaît consensuellement comme une avancée majeure. Il autorise l’arrêt des traitements de survie dans certaines circonstances et ouvre la possibilité de soulager la souffrance « même si » cela peut hâter la mort.  LE FIGARO. - Quels sont les risques liés à la proposition d’un nouveau projet de loi fin de vie dans la situation politique actuelle du pays ?  - Ce type de texte nécessite beaucoup de respect pour les positions de chacun et doit éviter les simplifications et les caricatures. Je ne suis pas sûr que le climat actuel dans l’Hémicycle soit favorable à une telle discussion. L’objectif du législateur d’aujourd’hui n’est pas de rechercher un équilibre entre le respect de l’autonomie et la protection de la vulnérabilité, mais d’ouvrir un « droit à la mort pour tous » assorti de quelques garde-fous pour rassurer les plus réticents. Mais, tôt ou tard, tous les verrous sauteront, comme cela s’est produit dans les pays qui ont légalisé l’euthanasie et le suicide assisté. Dans le texte présenté à l’Assemblée nationale, la collégialité médicale a disparu pour autoriser l’administration de la substance létale, la notion de pronostic vital engagé est remise en cause et certains députés, comme Sandrine Rousseau, veulent déjà ouvrir ce « droit » aux mineurs. Lorsque la boîte de Pandore sera ouverte, on ne pourra pas la refermer.  LE FIGARO. - Le premier ministre, François Bayrou, a voulu scinder le nouveau projet en deux textes, l’un sur les soins palliatifs et l’autre sur l’aide active à mourir. Est-ce une bonne idée ?  - Mettre en place un texte qui amalgamait artificiellement le développement des soins palliatifs et l’ouverture d’un droit à la mort était une manœuvre pour donner l’illusion d’un texte équilibré. Séparer les deux textes a le mérite de la clarté. Par ailleurs, le développement des soins palliatifs a plus besoin de moyens et de volonté politique que de textes législatifs.  - Pour la ministre de la Santé, Catherine Vautrin, l’objectif de la loi annoncée est de « garantir à chacun une fin de vie digne, dans le respect de son autonomie ». Ce que dit le gouvernement vous rassure-t-il ?  - Une fois de plus, on masque avec ce type de formule la réalité du projet. Il s’agit bien, en effet, de donner la mort, d’euthanasie, d’assistance au suicide. Ceux qui sont partisans de légiférer dans ce sens ne devraient pas se cacher derrière les formules creuses utilisées comme bouclier. Cette manœuvre, qui évite de dire ce que l’on va faire, en dit long sur la perception du caractère majeur de la transgression de donner la mort à l’autre dans notre société.  LE FIGARO. - Pourquoi soupçonnez-vous une offensive des pro-euthanasie derrière ce projet ?  - Je crois que c’est une évidence. Il n’y a pas dans ce texte d’autres objectifs. L’ouverture des soins palliatifs partout et pour tous ne peut pas être la compensation médiatique d’une loi sur l’euthanasie, elle doit au minimum en être le préalable, c - ar les Français doivent être égaux devant la loi et devant la mort. Personne ne conteste que les demandes de mort exprimées par les patients en souffrance et en fin de vie disparaissent lorsqu’ils sont pris en charge en soins palliatifs.  LE FIGARO. - LE FIGARO. - Où en sommes-nous concrètement en matière de développement des soins palliatifs en France ?  - La loi sur l’accès aux soins palliatifs pour tous date de 1999 et la situation est encore difficile. Certains centres sont même au bord de la fermeture parce qu’ils ont du mal à recruter des personnels.  LE FIGARO. - Au moment où les Pays-Bas, le Canada, l’Oregon ou la Suisse s’interrogent sur ces sujets après plus de vingt ans de pratique de l’euthanasie, quelle solution voyez-vous pour notre pays ?  - Ces territoires s’interrogent en voyant une progression importante du nombre de gens mourant par euthanasie ou suicide assisté. Peut-être que nous ferions bien d’écouter ce qu’ils disent et d’observer ce qu’ils font pour défendre une autre vision que celle du suivisme. La France est un pays de médecine et de soins qui a su défendre son originalité dans le domaine de l’éthique. À partir d’une réflexion approfondie et sur la base d’expériences internationales, notre pays peut proposer une troisième voie, une voie originale française, inventer des maisons d’accompagnement, imaginer de nouvelles approches des soins palliatifs, voire envisager la prise en compte de la spiritualité en fin de vie. Le moment n’est-il pas venu de nous interroger sur la place de la liberté et de la vulnérabilité dans nos sociétés modernes ?  LE FIGARO. - En tant que médecin, que répondez-vous aux personnes en souffrance et aux familles confrontées à certaines situations extrêmement difficiles ?  - Je leur dis que je sais la complexité et la difficulté d’aborder ces situations. Aucune ne ressemble à une autre et, la seule façon d’affronter l’insupportable, c’est d’avoir affaire à une équipe de soins compétente et humaine pour que les choses se passent le moins mal possible et de manière apaisée. Les cas les plus douloureux relèvent malheureusement le plus souvent d’une mauvaise compréhension ou application de la loi.  - Un sondage vient de révéler que les Français connaissent leurs droits en matière de santé, mais, en ce qui concerne la fin de vie, la chute du nombre de gens informés est vertigineuse. Près d’un sur deux ignore qu’en fin de vie et en cas de pronostic vital engagé à court terme, quand des souffrances réfractaires sont observées, il existe un droit à « dormir pour mourir sans souffrir ». C’est la loi de 2016. Chaque fois que nous avons essayé de faire la promotion des lois votées de manière consensuelle, le débat a été complètement confisqué par le choix récurrent et binaire sur l’euthanasie, nous imposant de choisir entre la souffrance et la mort. En vérité, il existe une troisième voie : celle de la non-souffrance et de la vie.   LE FIGARO. - LE FIGARO. - Pourquoi êtes-vous particulièrement sévère à l’égard du député Olivier Falorni (Dem, ex-PS), rapporteur général du texte ?  - Dans cette cause, des militants de l’euthanasie ont substitué le combat de la vie digne à la mort digne. Ils en ont fait presque une priorité politique. Généralement, ces gens sont issus de la gauche et défendent une forme de liberté individuelle alors qu’ils devraient placer la fraternité comme valeur prioritaire. Leur loi serait une loi laïque en opposition avec le « tu ne tueras point » de la loi divine. C’est ce que j’ai ressenti dans l’argumentaire du député Falorni. Il a raison sur la nécessaire laïcité de la loi, mais pour certains laïcs, comme moi, interdire de donner la mort à l’autre ne relève pas d’une ordonnance divine, c’est plutôt un impératif humain. Comme le dit le philosophe Emmanuel Lévinas, le visage de l’autre m’interdit de le tuer. Interrogé sur ce sujet, Robert Badinter n’a pas dit autre chose quand il estimait que, dans une démocratie, on ne donne pas la mort. L’hypothèse d’un duel entre Aulas et Doucet relance les municipales à Lyon  La rumeur d’une candidature et de premiers sondages flatteurs ont fait de l’ancien président de l’OL le champion providentiel du centre droit pour tenter de battre le maire écologiste sortant en 2026.   7 min • Justin Boche Antoine Sillières U n duel annoncé qui pourrait réveiller les municipales à Lyon. Largement critiquée, la majorité écologiste qui a repris la mairie à Gérard Collomb en 2020 avait néanmoins conforté son assise électorale lors des deux scrutins nationaux de l’an passé, raflant jusqu’à la 4e circonscription législative, taillée pour la droite depuis trois décennies. Le manque de consistance et d’incarnation de son opposition devait ouvrir un boulevard à la réélection de l’édile Grégory Doucet (EELV) qui a annoncé, dès la mi-mandat, briguer de nouveau la mairie.  L’apparition de Jean-Michel Aulas dans le champ politique local a donc été vue comme providentielle pour le centre et la droite, qui peinaient jusqu’ici à offrir une alternative crédible aux écolos. « Si Jean-Michel peut faire irruption dans le tissu local avec autant de force, c’est que le personnel actuel est dans les roses », commente un de ses proches. Ses tweets rageurs contre les travaux, les bouchons, ou l’insécurité ont peu à peu laissé grandir chez l’ancien patron de l’OL l’envie de s’engager dans l’arène politique. « Les écologistes crispent, et finalement la candidature de Jean-Michel Aulas est le réceptacle de ce mécontentement », prévenait récemment la députée socialiste et ex-adjointe de Doucet, Sandrine Runel.  Après avoir annoncé « réfléchir » à se présenter aux municipales dans les colonnes du Figaro, « JMA » a multiplié les rencontres. Chefs d’entreprise, élus locaux, nationaux. « Il discute avec tout le monde. Il a échangé avec Bruno Retailleau et Laurent Wauquiez, mais aussi avec des personnalités de gauche », confie son entourage. Jusqu’à aller chercher l’onction macroniste, à l’Élysée, le mois dernier. Très rapidement, les partis Renaissance, Horizons, le MoDem mais aussi plusieurs élus LR majeurs de la métropole, à l’image du ministre François-Noël Buffet, ont apporté leur soutien à sa candidature. Et délaissé au passage, le très actif maire du 2e arrondissement, Pierre Oliver (LR), aujourd’hui esseulé jusque dans son propre camp. Les appels de ce dernier à la démission du maire, après sa garde à vue dans l’affaire dite des « chargés de mission », ont ainsi été désavoués par des élus LR lyonnais de premier plan.  « Pour l’instant, certains montrent les muscles, mais tout le monde va se ranger s’il y va », confiait récemment un cadre Horizons local. Sans parti, sans appareil politique, l’actuel président de la Ligue féminine de football professionnel arriverait en 2e position lors du 1er tour des municipales dans presque toutes les configurations testées par les sondages. Il devancerait même Grégory Doucet de deux points dans une des hypothèses testées par Elabe pour BFMTV. Assez pour faire un match.  Plus politiquement, si « Lyon est une ville qui se gagne au centre droit », comme le martèle un autre élu Horizons de la métropole, « JMA » aurait tout intérêt, selon les enquêtes d’opinion, à ne pas endosser de casaque politique. « Il l’a très bien compris, assure un de ses conseillers. La condition même pour que sa candidature prospère, c’est de n’avoir qu’une seule étiquette : celle de Jean-Michel Aulas. » Là sera tout l’enjeu pour le chef d’entreprise : utiliser la force de frappe des partis sans se faire phagocyter par eux, tout en restant suffisamment puissant pour ne pas être trahi.  Les coups ont déjà commencé à pleuvoir   La percée d’Aulas est pour le moment observée sans grande inquiétude du côté de la mairie. « C’est prématuré d’anticiper quoi que ce soit parce qu’il n’est pas encore candidat. Toutefois s’il y va, ça permettra aux Lyonnais de trancher entre deux propositions très différentes pour la ville », assure-t-on du côté de l’hôtel de ville. Politiquement, la candidature de Jean-Michel Aulas pourrait même être utile à Grégory Doucet pour ressouder son camp et, face à la menace d’un adversaire commun, obliger le PS et LFI à laisser de côté leurs volontés d’indépendance.  Conscient de la rudesse du monde politique, l’homme aux sept championnats de France avec l’OL retarde, lui, pour le moment sa décision, expliquent ses proches : « Dès le début, il a dit que ce qui primait, c’est sa famille. Il a été ami avec des hommes politiques, et vu ce qu’ils se sont pris dans la gueule… C’est aujourd’hui un élément essentiel de sa réflexion. Il faut que sa famille se prépare à cette aventure. »  Déjà, quelques coups ont commencé à pleuvoir. « Est-ce que les Lyonnais vont vouloir d’un multimillionnaire de 76 ans qui fait des tweets rageux ? », a taclé dans LyonMag Valentin Lungenstrass, adjoint au maire de Lyon chargé des espaces publics. « Aulas avait du mal avec un seul arbitre, qu’est-ce que ça va être avec 500 000 à Lyon ? », cingle un autre écologiste auprès du Figaro.   Les contempteurs de Jean-Michel Aulas doutent également de sa capacité à gérer une mairie, à faire preuve de patience face à l’inertie d’une municipalité. Ses proches y voient « une forme d’arrogance du personnel politique à vouloir donner des leçons à quelqu’un dont il pense qu’il ne connaît pas l’univers public, alors qu’il a travaillé avec des élus de tous bords durant ses 36 ans à la tête de l’OL. »  Prompt à partager les articles critiques sur la gestion écologiste à Lyon, Jean-Michel Aulas n’a, cette fois-ci, pas commenté la récente garde à vue de Grégory Doucet, dans une enquête ouverte pour « détournement de fonds publics ». Une affaire dans laquelle a également été entendu l’ancien maire Georges Képénékian, aujourd’hui candidat centriste crédité de 10 % à 12 % dans les sondages. Une manière de ne pas insulter l’avenir pour « JMA ».  Avant de se faire face dans les urnes, Jean-Michel Aulas et Grégory Doucet se croisent pour le moment au bord des terrains de foot. Ils étaient presque côte à côte le 26 mars dernier, lors du quart de finale de l’OL féminin face au Bayern Munich. « Ça s’est très bien passé, le maire aime beaucoup Jean-Michel Aulas », assure l’entourage de l’édile écologiste. Au point même de comparer les deux hommes : « Le Aulas de 1999 (l’OL sera champion de France sept années de suite à partir de 2002, NDLR), c’est le Doucet d’aujourd’hui. Dans la phase de construction, mais les succès vont arriver. » Du côté de l’ancien patron, on préfère la référence Gérard Collomb. « Le destin de ces hommes est commun. D’ailleurs Jean-Michel ne revendique pas son héritage, il l’incarne, parce qu’il a fait partie de cette histoire et du rayonnement de Lyon. »  Après une guerre de mouvement ces dernières semaines, les positions semblent se figer entre Rhône et Saône, où chacun attend désormais la décision de l’ancien président de l’OL. Pour beaucoup, « JMA » a déjà posé trop de jalons pour ne pas se présenter. « Jean-Michel continue de consulter, de faire mûrir sa réflexion, d’engranger des soutiens, de travailler sur des bases programmatiques. Autour de lui, des équipes se mettent en place pour que, s’il décide d’appuyer sur le bouton, tout soit déjà près », indique un proche.  Il continue de dérouler sa feuille de route, méthodique, avec le cabinet parisien qui le conseillait déjà depuis plusieurs années. Si candidature il y a, elle sera annoncée au plus tard en septembre. Mais beaucoup attendent une décision pour la fin du printemps ou le début d’été. J. B. Louis Sarkozy lorgne la mairie de Menton   2 min • Lucas Hélin Claire Conruyt C ela fait déjà quelque temps que Louis Sarkozy s’intéresse sérieusement au parti des Républicains (LR), sa « famille politique », comme il aime à le rappeler. Son investissement dans le champ politique pourrait bien prendre un nouveau tournant. Selon nos informations, le fils du dernier président de la droite poursuit sa réflexion vers la Côte d’Azur et lorgne la mairie de Menton (Alpes-Maritimes). « Je ne suis candidat nulle part mais je ne peux pas dire que je ne m’intéresse pas à la politique de la région et notamment à Menton, qui est en passe de tomber dans les mains du Rassemblement national », confie Louis Sarkozy, bientôt 28 ans, au Figaro.  Une ville convoitée par le RN   À moins d’un an du scrutin prévu en mars 2026, les rumeurs de sa candidature aux municipales (et même aux législatives, en cas de dissolution prochaine) circulaient. Certains s’imaginaient une campagne à Neuilly-sur-Seine où Nicolas Sarkozy, fut élu maire en avril 1983. Ou encore une candidature dans les 8e et 16e arrondissements de Paris… La municipalité de Menton est actuellement dirigée par le maire divers droite Yves Juhel, dont le mandat est pollué par l’explosive affaire politico-financière de la gestion des ports de la ville. La députée Rassemblement national (RN) de la circonscription, Alexandra Masson, envisage ainsi de se présenter et aurait de sérieuses chances de l’emporter. Celle qui est aussi présidente de la fédération du RN des Alpes-Maritimes a confirmé qu’elle serait bien candidate aux prochaines municipales, sans préciser la ville, ce qui est un secret de Polichinelle.  Néanmoins, une candidature parachutée de Louis Sarkozy pourrait rebattre les cartes. « Il y réfléchit mais, lui, ne se rase pas », ironise un élu local qui pourrait bien le soutenir. « C’est une ville assez particulière », poursuit-il, expliquant qu’elle pourrait convenir à une candidature venue de l’extérieur, à l’inverse d’autres villes de Côte d’Azur comme Nice. Selon nos informations, Louis Sarkozy s’est procuré un appartement dans la cité du citron depuis quelques semaines. Du 13 au 17 mai, il sera le parrain de la Semaine du libraire organisée dans la ville aux façades colorées. Il doit y tenir une conférence sur son dernier livre consacré à Napoléon. Et déjà, plus aucune place ne serait disponible, soutient l’intéressé. L. H. À Paris, le candidat communiste Ian Brossat relance la guerre des voies sur berge   4 min • Richard Flurin À moins d’un an des municipales, la guerre parisienne des voies sur berge va donner lieu à une nouvelle bataille. Le sénateur communiste de Paris Ian Brossat, par ailleurs candidat déclaré à l’Hôtel de Ville, va déposer en fin de semaine prochaine une proposition de loi à la Chambre haute visant à « renforcer les compétences de la ville de Paris en matière de circulation et de piétonnisation des voies sur berge ». Un prérequis essentiel au projet phare de cet ancien adjoint au Logement d’Anne Hidalgo : piétonniser quasiment l’ensemble des voies sur berge parisiennes, c’est-à-dire interdire la circulation automobile sur la route qui jouxte la Seine du Pont du Garigliano (16e arrondissement) jusqu’au Parc de Bercy (12e).  La proposition de loi, si elle venait à être adoptée, lèverait le principal obstacle à la mise en place de ces voies sur berge XXL. Effectivement, une telle opération ne peut être réalisée sans l’aval de l’État, représenté à Paris par le préfet de police. Un premier article entend ainsi « conférer au maire de Paris la compétence exclusive en matière de circulation, d’aménagement et de piétonnisation des voies sur berge » dans le but de « mettre fin à l’intervention systématique du préfet de police dans la définition de l’usage de ces voies », lit-on dans l’exposé des motifs de ce texte que Le Figaro a pu consulter en exclusivité. La première phase de piétonnisation des voies sur berge, initiée dès 2010 par la mairie de gauche, a régulièrement achoppé sur la prudence, voire les réticences, des préfets de police de Paris. Seuls 3 kilomètres sont aujourd’hui piétonnisés, loin des 12 kilomètres envisagés par Ian Brossat.  « Je pense être le plus solide pour tenir tête à Rachida Dati »   Un second article du projet de loi porte sur une révision plus globale de la « répartition des responsabilités » entre les deux autorités, municipale et étatique. Il s’agit de « (reconnaître) au maire la compétence générale sur l’ensemble de la circulation et du stationnement à Paris », tout « en maintenant la capacité du préfet à intervenir temporairement pour des motifs impérieux de sécurité ». Une disposition qui laisse penser que la volonté de piétonniser dépasse largement les seules voies attenantes à la Seine. International  Ces Chinois à l’épreuve de la guerre en Ukraine  Ces mercenaires seraient plusieurs centaines à combattre avec les forces russes, et une poignée du côté de Kiev, selon des témoignages.   7 min • Sébastien Falletti D epuis son Shandong natal, « Macaron » rêvait de guerre et du képi de la Légion étrangère. Le jeune chinois a finalement connu l’épreuve du feu parmi les ruines de Bakhmout, dans les rangs de l’armée russe. Cet ancien soldat de l’Armée populaire de libération (APL), l’armée chinoise, au nom de guerre improbable, a déchanté. « Je n’avais jamais connu le combat. Beaucoup regardent des films de guerre patriotiques et deviennent tout excités. Mais la réalité du combat, c’est comme l’enfer sur terre », explique le jeune homme en uniforme au visage poupin, enfoncé sous un bonnet couleur camouflage. En bruit de fond, des détonations d’artillerie, « à cinq ou six kilomètres » de la ligne de front.  Cette tête brûlée est l’un des mercenaires interviewés dans le documentaire Êtes-vous de la chair à canon ? de Chai Jing, ancienne journaliste star de la télévision chinoise, désormais indépendante et qui produit ses propres reportages hors du système médiatique chinois. Le documentaire a été posté le 15 mars dernier sur la plateforme YouTube.  « Macaron », donc, raconte avoir lui-même imaginé des exploits guerriers en regardant Wolf Warrior, blockbuster patriotique orchestré à la gloire de l’armée chinoise qui a décroché un record au box-office en Chine en 2015, symbolisant l’ère décomplexée des « loups guerriers » de Xi Jinping. « Mon objectif était de rejoindre la Légion étrangère, mais comme il est difficile d’obtenir un visa pour la France, j’ai opté pour la Russie », explique le combattant, arrivé avec un visa de tourisme chez le grand voisin eurasiatique.  Après avoir signé un contrat avec le ministère de la Défense russe en mai 2024, pour une solde de 2 500 dollars par mois, il est formé à la va-vite avant d’intégrer une « unité d’assaut » chargée d’infiltrer les lignes ukrainiennes, aux côtés des mercenaires du groupe paramilitaire privé Wagner.   La présence de combattants chinois dans les rangs russes a été confirmée par la capture sur le front de deux soldats chinois par Kiev, lors de combats près de Donetsk, le 8 avril et dénoncée par Volodymyr Zelensky. Une nouvelle pomme de discorde entre Kiev et la deuxième puissance mondiale. Les soldats chinois seraient « plusieurs centaines » sous uniforme russe affirme le président ukrainien, dénonçant un appui « direct ou indirect » de Pékin à l’effort de guerre de Vladimir Poutine. Des « propos irresponsables », a répliqué le régime communiste, embarrassé par cette découverte. Pékin affiche une neutralité de façade depuis le début du conflit, mais n’a jamais condamné l’invasion lancée par Poutine quelques jours après avoir conclu un « partenariat sans limite » avec Xi Jinping, en ­février 2022.   « Ne participez pas à cette guerre ! » a lancé Wang Guangjun, 33 ans, l’un des deux soldats capturés, à l’adresse des candidats au front, le 14 avril lors d’une conférence de presse à Kiev, où les prisonniers de guerre sont apparus menottés. Lors de cet événement médiatique organisé par Kiev, et lors duquel il est impossible de savoir s’ils pouvaient s’exprimer librement, il a appelé à un échange de prisonniers avec Moscou, dans l’espoir de regagner son pays. Ces mercenaires affirment avoir répondu à des annonces de recrutement postées sur les applications comme Douyin, la version chinoise de TikTok.  Alors que la Corée du Nord du Leader suprême, Kim Jong-un, a déployé plus de 10 000 de ses soldats sur le front ukrainien, sans jamais l’admettre officiellement, après un traité de défense mutuel conclu avec le Kremlin en 2024, la Chine n’a pas franchi ce pas. Aucune trace de coordination étatique n’a été avérée. « Il existe visiblement des circuits de recrutement organisés par des sociétés de sécurité, mais rien de comparable avec l’ampleur de Wagner », juge un ancien militaire, fin connaisseur de l’APL.  La propagande patriotique dénonçant « l’encerclement » américain et la nouvelle fraternité sino-russe nourrit des vocations individuelles au regard de multiples témoignages de combattants chinois postés sur la toile chinoise. De jeunes hommes visiblement mus par un élan patriotique, l’envie d’en découdre ou des motivations sonnantes et trébuchantes, à l’heure du ralentissement de la croissance… « Beaucoup pensent qu’aider la Russie revient à aider la Chine » explique « Macaron » dans le documentaire. Ces combattants se comptent par « centaines » juge le soldat aujourd’hui blessé. « La Russie et la Chine sont dans le même bateau » justifie un autre mercenaire interviewé. « Non à l’expansion de l’Otan » ajoute un troisième, reprenant les éléments de langage des régimes autoritaires qui désignent l’Occident comme les responsables du conflit.  Les soldes promises par les recruteurs sont alléchantes. « Je suis loyal à celui qui me paye ! Si je rentre, qui va me nourrir ? » lance le combattant « Warden » sur la plateforme Bilibili. « Beaucoup sont endettés jusqu’au cou et n’ont plus rien à perdre », confirme « Macaron ». Mais la paie est loin d’être mirobolante une fois rapportée aux coûts de la vie au front, ajoute le mercenaire qui a dû acheter son casque et une partie de son équipement avant de se voir fournir un fusil datant de 1948…  L’âpreté du front et l’attitude méprisante de certains officiers russes ébranlent les Chinois envoyés en première ligne. « Les Russes ne nous voient pas comme des êtres humains, mais des escadrons-suicide. On nous avait ordonné de prendre des tranchées, mais faute de renfort nous avons enregistré de lourdes pertes. Sur plusieurs dizaines, seuls six ou sept d’entre nous sont rentrés indemnes », raconte Zhou Zhiqiang, qui date son baptême du feu au 11 novembre 2023. « Macaron » dénonce le « racisme » d’officiers russes à l’encontre « des Noirs, des Arabes et des Chinois ».  Les motivations idéologiques ont leur limite. « Macaron » reconnaît qu’il aurait pu tout aussi bien lutter côté ukrainien si les circonstances l’avaient permis : c’est la facilité à entrer en Russie avec un visa de tourisme qui l’a conduit dans les rangs de Moscou.  Une poignée de Chinois a pourtant rejoint les rangs ukrainiens, selon un autre documentaire produit par Chai Jing, posté sur YouTube le 30 mars. Parmi eux, un ancien de la Légion étrangère, surnommé « Gao Shan », jure, visage caché, avoir volé au secours de Kiev dès l’été 2022. Armé de sa propre mitraillette, il a rejoint la compagnie A de la « Légion internationale pour l’Ukraine », une unité rassemblant des volontaires du monde entier. Un engagement nationaliste à rebours de la position de Pékin, et issu d’une méfiance historique à l’égard de la Russie. « La Russie a occupé de nombreux territoires chinois comme la Mongolie ou encore Vladivostok, et elle continue à le faire en agressant d’autres pays. Aujourd’hui, la Chine ne dit rien pour des raisons diplomatiques », accuse « Gao Shan ».   D’autres sont des idéalistes, comme cet ingénieur de l’unité des drones de la 110e brigade ukrainienne, qui combat la Russie avec un drapeau chinois agrafé sur son uniforme depuis juillet 2024. Un engagement rare suscité par l’émotion, après avoir appris des frappes de missiles contre des hôpitaux pédiatriques à Kiev ce mois-là, rapporte la productrice Chai Jing.  Désormais blessé, dans un hôpital à l’arrière du front, « Macaron » est revenu de ses rêves de « loup guerrier » et attend avec impatience la fin de son contrat en mai pour rentrer en Chine. Tirant les leçons de son aventure russe, il offre indirectement des conseils de prudence au président Xi, champion d’une renaissance nationaliste décomplexée, et qui exhorte l’APL à « gagner des guerres », avec Taïwan en ligne de mire. « J’espère que la Chine va maintenir une position sage en se retenant de lancer une guerre ou d’être aspirée dans une autre » déclare le mercenaire repenti.  S.F. International  Giorgia Meloni gagne la bienveillance de Donald Trump, au prix fort  Cajolée par le président et le vice-président américains, entre louanges et promesse de visite, la chef du gouvernement italien a dû consentir à des engagements, notamment commerciaux, très avantageux pour Washington.   4 min • Segond, Valérie L a visite de Giorgia Meloni à Donald Trump, suivie hier par la venue à Rome du ­vice-président JD Vance, n’aurait pu mieux se ­passer. À Washington d’abord, le président américain a couvert son invitée de louanges personnelles lors de la conférence de presse : « Meloni, un des véritables leaders mondiaux », « Meloni devenue une amie », « qui fait un excellent travail en Italie. » « Sur le plan médiatique, cette rencontre est un vrai succès » résume Emidio Diodato, professeur de relations internationales à l’université pour étrangers de Pérouse.  Mieux encore : Giorgia Meloni ne sera pas revenue à Rome les mains vides. Répondant à son invitation, le président américain a accepté de venir prochainement en visite officielle en Italie. Et alors que la chef du gouvernement italien avait proposé d’élargir l’événement à l’Europe, les deux ont fait savoir dans un communiqué commun hier que cette possibilité « sera à l’étude. » « La venue du président américain à Rome est le maximum que Giorgia Meloni pouvait espérer, explique Emidio Diodato. Un résultat qui s’inscrit dans la grande tradition de la “politique du siège” de la diplomatie italienne, qui consiste à organiser des sommets internationaux. »  Si la présidente du Conseil des ministres d’Italie n’avait pas de mandat explicite de l’Europe pour discuter des droits de douane américains à l’encontre des produits européens, le sujet était bien au menu de la rencontre. Donald Trump a certes affirmé ne pas avoir changé d’avis sur les « droits de douane qui nous enrichissent ». Mais il s’est déclaré prêt à négocier, se disant certain « à 100 % qu’il y aura un accord global ». Dans leur communiqué commun jeudi, Rome et Washington se présentaient comme étant « d’accord pour œuvrer afin que les échanges commerciaux entre les États-Unis et l’Europe soient mutuellement avantageux, équitables et réciproques. »  Au même moment, JD Vance affirmait à Rome avoir « évoqué les relations économiques non seulement entre l’Italie et les États-Unis, mais aussi avec l’Europe dans son ensemble ». Comme ultime geste de coopération, le vice-président américain déclarait aussi avoir « informé la présidente de l’état d’avancement des négociations entre la Russie et l’Ukraine, et lui avoir fait part en privé des dernières nouvelles de ces dernières 24 heures. Nous es­pérons pouvoir mettre fin à cette guerre brutale. »  Cette relation privilégiée a un coût, de plus en plus perceptible. « Nous soulignons l’importance des technologies de l’information pour permettre la libre entreprise de part et d’autre de l’Atlantique, affirme ainsi le communiqué de jeudi. Nous avons convenu qu’un environnement non discriminatoire en matière de taxation des services numériques est nécessaire pour permettre les investissements des entreprises technologiques de pointe. Nous saluons les investissements américains dans l’intelligence artificielle et les services cloud en Italie. » « Giorgia Meloni a-t-elle servi de cheval de Troie en Europe pour les entreprises américaines de la Big Tech ? », se demande le député Renew Sandro Gozi.  De fait, le communiqué commun illustre tout ce à quoi la présidente du Conseil italien a dû consentir en échange de la bienveillance de Donald Trump. À commencer par une augmentation des importations de gaz liquéfié en provenance des États-Unis. Il y a aussi un accord d’intégration économique, reliant l’Inde au Golfe, en passant par Israël, l’Italie et jusqu’aux États-Unis, et qui vise à s’opposer au projet chinois de « route de la soie ». Toujours pour contenir l’expansion chinoise, les États-Unis s’engagent à investir dans des projets d’infrastructures en Afrique en coordination avec le plan Mattei de Giorgia Meloni. Enfin, précise le communiqué, « reconnaissant la nécessité de protéger leurs infrastructures et technologies nationales critiques et sensibles », l’Italie s’engage avec les États-Unis « à ne faire appel qu’à des fournisseurs de confiance dans ces réseaux ».    En outre, « il n’y a pas de confiance plus grande que notre alliance stratégique, c’est pourquoi il ne peut y avoir aucune discrimination entre les fournisseurs américains et italiens ». L’entreprise Starlink d’Elon Musk attend impatiemment de signer le contrat de 1,5 milliard d’euros sur les communications sensibles des ambassades et de la protection civile italiennes. Enfin, il est fait mention d’un « renforcement de partenariats dans les technologies du ­futur », comme le réseau télécom 6G, l’intelligence artificielle, l’informatique quantique et les biotechnologies. L’Italie s’est livrée aux géants américains. International  JD Vance, vice-président américain sur tous les fronts du trumpisme   7 min • Adrien Jaulmes L e vice-président JD Vance est sur tous les fronts. Le vice-président américain, récent converti au catholicisme, sera reçu au Vatican en pleine semaine sainte, pour y rencontrer le cardinal secrétaire d’État Pietro Parolin et, peut-être, le pape. Après un entretien vendredi avec Giorgia Meloni, l’une des rares personnalités européennes à trouver grâce aux yeux du mouvement Maga, Vance continuera son voyage vers l’Inde, partenaire important des États-Unis face à la Chine, et aussi pays d’origine de sa femme.  Alors que Donald Trump n’a encore effectué aucun déplacement à l’étranger, le premier étant programmé pour la mi-mai au Proche-Orient, son vice-président joue le rôle de porte-parole international de la nouvelle administration, en même temps que celui de grenadier-voltigeur du nationalisme Maga. Avant de le manifester par sa critique publique du président ukrainien Volodymyr Zelensky dans le Bureau ovale, Vance avait annoncé le nouvel ordre mondial dans un discours à la Conférence annuelle de Munich sur la Sécurité. « Il y a un nouveau shérif à Washington », avait annoncé Vance. Il avait accusé les Européens de tourner le dos à leurs propres valeurs, en réprimant selon lui les discours populistes au nom de la lutte contre la désinformation. Et avait ironisé sur « l’idée que quelqu’un ayant un point de vue différent puisse exprimer une opinion différente ou, Dieu nous en garde, voter différemment, ou pire encore, remporter une élection ».  Vance ne se prive pas d’exprimer ses opinions. Au lieu du rôle plutôt effacé habituellement réservé aux vice-présidents américains, entre obsèques officielles et missions non prioritaires, JD Vance sert de porte-voix de choc à Donald Trump. Comme lorsqu’il est allé prononcer un discours provocateur sur la base polaire américaine de Pittuffik, revendiquant le Groenland depuis le territoire danois.  Au lieu de la vie en retrait que menait Mike Pence, le premier vice-président de Trump, Vance et son épouse se montrent en public. Ils bravent parfois les sifflets et les huées, comme lors d’un concert au Kennedy Center à Washington ou pendant un week-end aux sports d’hiver dans le Vermont, où des manifestants locaux brandirent des panneaux l’encourageant à aller plutôt « skier en Russie ».  Installé à l’Observatoire Naval, la résidence des vice-présidents, l’ancien « petit blanc pauvre » élevé par ses grands-parents dans les Appalaches, le caporal des Marines dans le Triangle sunnite devenu rétif aux aventures militaires, le diplômé de la faculté de droit de Yale, auteur à succès à moins de trente ans et investisseur dans la haute technologie, continue sa trajectoire hors du commun.  Propulsé à la vice-présidence après une carrière politique d’à peine deux ans comme sénateur de l’Ohio, Vance est devenu l’héritier présomptif du mouvement Maga. Trump l’a nommé à la tête des finances du Parti républicain, position clé des futures campagnes où se constituer un carnet d’adresses de riches donateurs. Mais Vance est aussi l’intellectuel de la nouvelle administration, capable d’articuler dans une idéologie un tant soit peu cohérente des politiques parfois contradictoires. Notamment de faire le lien entre le populisme nativiste de la base Maga et le libertarisme dérégulé et accélérationiste de la haute technologie. Cette synthèse n’est pas seulement intellectuelle, mais aussi personnelle, Vance incarnant lui-même le maillon entre les différentes factions de la coalition rassemblée par Trump.  Il est l’une des figures de l’aile isolationniste et souverainiste du mouvement. Issu de la Nouvelle droite, courant d’intellectuels et activistes néoréactionnaires rompus aux batailles culturelles sur les réseaux sociaux et dans les podcasts, Vance défend le protectionnisme économique, l’opposition virulente à l’immigration et s’oppose aux engagements extérieurs américains. Il est proche des principaux représentants de ce courant, comme Donald Trump Jr., le fils aîné du président, qui occupe dans le second mandat le rôle de principal conseiller, et Tucker Carlson, l’influent ex-présentateur de Fox News dont les podcasts servent de chronique à la nouvelle administration ? Tous deux ont soutenu sa candidature à la vice-présidence. Il combat avec eux dans les guerres culturelles contre le « wokisme ». Il se méfie comme eux des républicains traditionnels ralliés à Trump, soupçonnés de reaganisme, partisans de fortes institutions fédérales et du rôle international des États-Unis.  Mais Vance joue aussi le rôle d’intermédiaire entre la nouvelle administration et le Congrès, où siègent encore de nombreux représentants de cette vieille garde républicaine. Son vote en tant que vice-président a servi à confirmer au Sénat la nomination au Pentagone de Pete Hegseth, rejetée par plusieurs sénateurs républicains, mais que Vance était venu personnellement soutenir.  Et surtout, le « Veep » continue d’incarner le lien avec les alliés de la dernière heure de l’Administration Trump : le monde de la haute technologie californienne, auquel il a appartenu et dont il est resté proche. En particulier de son mentor, le milliardaire et investisseur Peter Thiel, l’un des fondateurs de PayPal et l’un des précurseurs du virage idéologique de la Silicon Valley vers un technocratisme alliant conservatisme social et libertarisme dérégulateur et antiétatique.  Le démantèlement de pans entiers de l’État fédéral par Elon Musk, qui met mal à l’aise de nombreux élus républicains en butte aux protestations de leurs électeurs, a été soutenu par Vance.  « Fondamentalement, Elon restera un ami et un conseiller du président et de moi-même, et il a fait beaucoup de bonnes choses », avait déclaré Vance après l’annonce du départ prochain de Musk du gouvernement. Alors que les critiques les plus acerbes contre Musk étaient venues de Steve Bannon, autre figure populiste Maga, Vance a minimisé ces tensions. « En tant que membre fier des deux tribus », avait lancé Vance dans un discours prononcé le mois dernier lors d’une conférence à Washington, « l’idée que les gens à l’avant-garde de la technologie et les populistes doivent inévitablement s’affronter est fausse ». Citant l’encyclique du pape Jean-Paul II, « Laborem exercens », Vance a expliqué que « la technologie devrait être quelque chose qui renforce la valeur du travail plutôt que de le supplanter ». Condamnant la « surréglementation » comme un « étouffement de l’innovation », il a tenté de faire la synthèse entre les deux en arguant de ce que « chaque groupe - nos travailleurs, la population d’une part, les optimistes de la technologie d’autre part - a été déçu par le gouvernement ces quarante dernières années ».  Vance est capable de la même gymnastique intellectuelle sur d’autres sujets. Représentant le courant le plus antieuropéen du mouvement Maga, position confirmée dans l’affaire du SignalGate : son intervention dans cet échange interne à l’administration divulgué par un journaliste inclus par mégarde, avait été pour critiquer les frappes contre les houthistes du Yémen comme allant surtout bénéficier aux Européens.  Dans un récent entretien accordé au média britannique Unherd, JD Vance a affirmé « aimer l’Europe et aimer les Européens », tout en expliquant que ses critiques n’étaient qu’un appel à prendre en main leur propre destin. « Je me fais l’écho d’un grand homme d’État européen, Charles de Gaulle, qui aimait les États-Unis d’Amérique tout en ayant évidemment des désaccords avec eux », avait dit Vance, « et qui avait reconnu… qu’il n’est pas dans l’intérêt de l’Europe ni dans l’intérêt de l’Amérique que l’Europe soit un vassal permanent des États-Unis en matière de sécurité ». A. J. Société  Cette vague de jeunes fidèles qui déferle sur  l’Église catholique  Le carême, qui conduit à la fête de Pâques, a révélé partout dans le pays un engouement inattendu pour la foi de la part d’une nouvelle génération pourtant éloignée de la religion.   12 min • Jean-Marie Guénois E va Becue, 17 ans, est en classe de terminale dans un lycée de Villepreux (78). Elle sera baptisée au cours de la nuit pascale, ce samedi saint, dans sa paroisse. Quelques jours avant le grand rendez-vous, la jeune fille était à Lourdes, au milieu de 13 500 jeunes des diocèses d’Île-de-France. Ensemble, ils vivaient le « Frat », abréviation de « fraternel », une rencontre organisée dans la cité mariale tous les deux ans pour les lycéens franciliens. « Je suis prête, confie-t-elle à propos de ce baptême tout proche. Je n’attends qu’une chose, c’est d’entrer dans l’avion ! » L’avion ? L’Église catholique, pour cette jeune fille prête à s’élever après avoir mis un genou à terre.  « Il y a deux ans, je n’allais pas bien, ma vie était compliquée. J’ai tout essayé dont la psychologie. Rien n’a marché, je n’allais pas mieux », explique la lycéenne aux longs cheveux bruns. Un matin, tout a changé : « Je me suis réveillée et je me suis dit : “Je veux suivre Jésus. C’est maintenant !” » Elle entre alors dans des églises, achète une bible en bande dessinée, qu’elle dévore, se « renseigne » sur le christianisme sur internet et finit par se rendre compte qu’elle a pas mal d’amis « croyants catholiques ». Ceux-ci l’encouragent à venir à l’aumônerie du lycée. Elle qui n’est pas d’une famille croyante se décide à franchir le pas : « Jésus m’a sauvée. Je voulais apprendre à le connaître. J’ai alors demandé à me préparer au baptême. »  Autour d’Eva, dans la salle à manger de l’hôtel La Solitude, qui porte bien mal son nom ce jour-là, des centaines de jeunes finissent des spaghettis à la bolognaise dans une ambiance de cantine scolaire. À la différence que cet établissement hôtelier est à deux pas de la grotte des apparitions et que le Frat, aussi festif soit-il, est une retraite spirituelle. Après quatre jours à Lourdes, Eva, la jeune catéchumène (du nom des baptisés volontaires, adolescents ou adultes), conclut sans hésitation : « La joie de ce rassemblement m’a confirmée dans ma décision de recevoir le baptême : c’est bon, je suis prête. J’ai connu la dépression. Je me suis accrochée à Jésus. C’est le seul qui tire vers le haut. Je suis heureuse ! »  Ébranlée par les scandales, encore mis en lumière cette semaine par les dernières révélations sur l’abbé Pierre, l’Église de France connaît aussi une actualité heureuse : un phénomène d’adhésion massif et volontaire se produit depuis deux ans, au point que l’Église a du mal à gérer l’afflux de demandes de baptêmes d’adolescents et de jeunes adultes.  Quelques chiffres objectifs, tous vérifiables, en donnent la mesure. Ce samedi soir, une fois le grand feu pascal allumé sur les parvis des églises, il y a aura en France 10 384 baptêmes d’adultes. Il y en avait 5 423 en 2023. À ce chiffre s’ajoute celui des baptêmes d’adolescents entre 12 et 18 ans, comme celui d’Eva, qui connaissent une progression encore plus spectaculaire : ils étaient 2 953 en 2023, ils seront 7 404 en 2025. 17 788 adolescents et adultes seront ainsi baptisés, à leur demande, dans l’Église catholique cette année. Ce phénomène est certes lié à la chute drastique des baptêmes à la naissance - un enfant sur deux en l’an 2000, un sur trois en 2024 -, mais rien n’oblige aujourd’hui ces ados et ces adultes à pousser les portes des églises.  Autre indice : les évêques de France, réunis à Lourdes en assemblée de printemps début avril, ont tous, sans exception, noté une affluence record pour la messe des Cendres ouvrant le carême il y a quarante jours. Vice-président de la conférence épiscopale, Mgr Vincent ­Jordy, archevêque de Tours, racontait avec des étoiles dans les yeux, comme beaucoup de ses confrères : « J’avais devant moi une cathédrale pleine à craquer avec plus de 1 000 jeunes de moins de 30 ans que nous ne connaissions pas et dont nous ne savions pas comment ils étaient arrivés là. »  Ancien recteur de l’Institut catholique de Lyon, aujourd’hui président recteur délégué de l’Institut catholique de Lille, docteur en théologie, le père Thierry Magnin relate, pour sa part : « 1 400 jeunes se sont présentés aux deux célébrations des Cendres organisées par l’université. Un bon nombre est venu à la procession de communion les bras croisés, signe qu’ils sont néophytes et n’ont pas fait leur première communion. La fréquentation était au moins 30 % supérieure en 2025 par rapport aux années précédentes. Ce fut une grande joie d’être témoin de cette quête spirituelle forte. » Le diocèse aux armées, sous la conduite de Mgr Antoine de Romanet, qui organise un pèlerinage à Lourdes du 16 au 18 mai prochain, voit lui aussi ses inscriptions « bondir de 30 % cette année, avec près de 7 000 inscrits et 200 baptêmes prévus, alors que la moitié des participants ont entre 18 et 30 ans. »  Autre exemple, avec 12 000 inscrits en 2022, 16 000 en 2023, 18 000 en 2024, le pèlerinage de Chartres prévu en juin prochain, de sensibilité traditionaliste, a dû suspendre les inscriptions pendant une semaine, quelques jours seulement après leur ouverture pour l’édition 2025, le temps d’absorber une affluence record des demandes « deux à trois fois supérieures à celles de 2024 » selon l’organisation. Enfin, autre indice convergent, les inscriptions au Frat de Lourdes ont augmenté de 55 % en deux ans pour atteindre 13 500 jeunes cette année. « C’est la capacité de l’immense église souterraine de Lourdes, la basilique Saint-Pie-X, et la capacité hôtelière de la ville qui nous ont contraints de bloquer les inscriptions. Sans cela, nous aurions pu rassembler 20 000 jeunes », garantit le père Gaultier de Chaillé, responsable du Frat, épuisé mais heureux à la fin de ces intenses journées lourdaises.  Comment expliquer ce qui n’est pas un « réveil » - cela supposerait que les jeunes chrétiens se seraient endormis - mais un « éveil » spirituel inédit dans l’Église de France ? Un éveil accompagné par une quête insatiable de spiritualité constate le père Gaultier de Chaillé, comme beaucoup de prêtres présents : « Cette génération, qui n’est jamais en silence, attend du silence. Le silence de la prière intérieure ne lui fait pas peur. » Il reconnaît que la proposition liturgique du Frat s’est adaptée : « On est passé du crypto-catho, où il ne fallait pas trop parler de Dieu, au catho cash, explicite ! C’est ce que veulent les jeunes. »  De fait, il fallait « entendre », mardi soir, après une entrée en matière très rock, le silence qui s’est imposé lors de l’entrée d’un ostensoir, porteur d’une large hostie consacrée, dans la basilique, ouvrant un long temps d’adoration conduit par Mgr Matthieu Rougé, évêque de Nanterre. « Ils sont une génération très identitaire, sans arrière-pensées, explique le père Gaultier de Chaillé. Dans les lycées, ils sont face à l’affirmation sereine des jeunes musulmans qui n’hésitent pas à dire : “Je suis musulman”, ce qui les place maintenant face à leurs propres racines chrétiennes. Et l’indifférentisme religieux de leurs parents leur donne, paradoxalement, la possibilité d’une adhésion choisie, mûrie, très personnelle. »  Ambroise Gambotti, 16 ans, élève d’un lycée de Saint-Gemain-en-Laye, confirme : « On vit dans une société où la religion est de plus en plus présente. Cela nous permet de poser des choix libres et individuels. » Son ami Romain d’Ersu, 17 ans, scolarisé dans un lycée public, assure : « J’ai des amis musulmans qui savent que je suis chrétien. On parle de nos points communs, mais on s’explique sur nos différences. La religion est acceptée, ce n’est plus du tout un sujet tabou. »  Curieux, tout de même, que ces nouveaux jeunes catholiques fleurissent sans qu’aucune promotion particulière n’ait été réalisée par l’Église pour les attirer et alors que les scandales d’abus sexuels ont tout d’un repoussoir.  Née avec ces affaires, cette génération n’y est pas indifférente, mais elle fait la part des choses entre la foi et l’institution. Issus d’une même aumônerie près de Créteil, Tyvan Emono, 16 ans, d’origine africaine, Tanita Aubrée, 17 ans, dont les deux parents sont des chrétiens évangéliques et qui a demandé le baptême dans l’Église catholique, et Léa ­Magnol-Bouzy, 16 ans, bouclent leurs bagages avant de remonter en Île-de-France. « Ces scandales ne gênent pas ma foi, confie cette dernière. Il y a des gens bons et des gens mauvais dans tous les domaines. »  Tyvan ajoute : « Dans ces scandales, ce n’est pas la religion qui est le problème, ce sont les personnes qui abusent en utilisant la religion. Ces affaires ne sont pas à cause de la religion, il y a du bien et du mal partout, ce ne sont pas des bons exemples. » Sophie Lazzarin, leur accompagnatrice, reconnaît : « Les jeunes n’en parlent jamais. Les générations précédentes sont davantage interpellées par ces scandales. »  Que se passe-t-il donc sur la planète catho ? Clément Bluteau, 28 ans, a animé le Frat avec son groupe Be Witness (« soyez des témoins ») et leur musique de rock chrétien, tonique et limpide en termes de louanges et d’évangélisation. La bande connaît un vrai succès, avec un agenda de concerts quasi complet jusqu’en mai 2026. Au contact régulier de cette nouvelle jeunesse catholique qui connaît ses chansons par cœur, Clément Bluteau explique : « Contrairement à la génération précédente, qui a pu être dégoûtée par l’Église, ces jeunes n’ont plus aucun a priori, tout simplement parce qu’ils n’ont rien reçu. Il n’y a même plus la grand-mère à qui on faisait plaisir en baptisant les petits. Ils n’ont eu aucun catéchisme. Nous sommes comme les premiers chrétiens face à des gens qui ne connaissent pas Jésus. On voit Dieu à l’œuvre qui donne le feu à des jeunes qui n’ont plus peur de dire qu’ils croient en ­Jésus ! »  Sœur Bernadette, 86 ans, est la franciscaine qui a été reconnue 70e mira­culée de Lourdes en 2018 pour une guérison inexpliquée du syndrome paralysant de la « queue de cheval » dix ans plus tôt. Parmi les invités du Frat, elle a témoigné devant quatre groupes de plus de 3 000 jeunes. « J’ai déclenché un tonnerre d’applaudissements quand j’ai dit pourquoi j’étais contre l’euthanasie, s’enthousiasme-t-elle. Car rien n’est jamais perdu, une vie aussi cabossée qu’elle soit, Dieu peut toujours la retourner, jusqu’à la fin ! » Cette religieuse passe une bonne partie de sa vie à rencontrer des jeunes dans toute la France. Elle est frappée par l’atmosphère de violence dans laquelle baigne cette jeune génération : « On ne réalise pas combien ils ont déjà une vie heurtée par la violence, à l’école, dans leur famille, vous n’imaginez pas les confidences que je reçois… Face à cette violence, les jeunes ont besoin de se raccrocher à l’essentiel. Isolés, ils cherchent une fraternité. Perdus, ils cherchent un sens à leur vie. Depuis deux à trois ans, je sens des jeunes qui ont soif, soif de prière, soif de Dieu. »  Jeune prêtre en baskets et en soutane, aumônier de jeunes de la paroisse Saint-Pierre-de-Montrouge dans le 14e arrondissement de Paris, le père Martin de Laubadère confirme : « Il y a une immense quête de sens chez les jeunes, qui, pour beaucoup, trouvent Dieu en se cherchant eux-mêmes, mais c’est Dieu qui les trouve. Ce réveil ou cet éveil vient de Dieu, et non de ce que l’Église a pu faire. La moitié des cinquante jeunes de mon groupe sont arrivés dans l’Église par leurs amis, qui osent leur parler de la foi. Il y a un décalage entre les sommations du monde - “Achetez, consommez !” - et ce qui se passe sur les réseaux sociaux, où la question religieuse s’est imposée, alors qu’elle est récusée dans l’espace public. Il apparaît que les jeunes ont besoin de silence, de cœur à cœur avec Dieu. »  Il marche vite pour ne pas être en retard. Évêque auxiliaire de Paris, Mgr Philippe Marsset va animer dans quelques minutes la session finale du Frat devant les 13 500 jeunes. « J’ai l’habitude de prêcher, mais reconnaissez que c’est impressionnant », glisse-t-il. Surtout, il confirme : « 8 % à 10 % des jeunes rassemblés ici se préparent au baptême ou ne sont pas chrétiens. C’est considérable. Cela veut dire que nos réseaux cathos sont dépassés par les réseaux sociaux. Tout cela est rapide, il y a un effet boule de neige. Mais il y a une raison plus spirituelle, que saint Paul avait développée : “Là où le péché abonde, la grâce surabonde.” Notre Église, indigne des scandales que l’on découvre, fait cette expérience que la grâce du Seigneur est plus importante, plus fé­conde que le péché. C’est un peu une résurrection après une mort. C’est tout le mystère de Pâques. » J.-M. G. Débats  « Quiconque se prétend religieux a le devoir de questionner son rapport à l’Écriture et à la terre »  OLIVIER-THOMAS VENARD Dans « Il nous reste la foi », le frère dominicain à Jérusalem débroussaille la voie d’une authentique recherche de Dieu au milieu du désastre physique et moral qui désole la Terre sainte. Une quête que Pâques nous invite à renouveler.   11 min • Eugénie Boilait LE FIGARO. - Vous commencez votre ouvrage en évoquant une transmission familiale de la foi « par le rite et par l’esprit critique ». Pourquoi cette insistance ?  OLIVIER-THOMAS VENARD. - Pendant longtemps, j’ai trouvé pesant d’être né dans une famille où il y a des ecclésiastiques et des militaires à toutes les générations depuis la Révolution française… Heureusement, je suis le premier dominicain de toute la lignée - mon père me l’a fait remarquer, pendant mon noviciat : je ne suis donc pas totalement prédéterminé !  En vieillissant, je me suis rendu compte que c’était en réalité une bénédiction, particulièrement dans le monde explosé d’aujourd’hui. Et même une double bénédiction. Car il y avait d’un côté le « rite », c’est-à-dire la messe du dimanche, le bénédicité avant le repas, la prière en famille, où l’on demandait pardon, on rendait grâce au Seigneur, on priait aux intentions du monde… Loin d’enlever notre liberté intérieure, ça lui donnait un cadre pour se développer, tantôt en plein accord, tantôt en opposition, mais cela faisait partie du rythme de la maison et, tant que tu vivais sous ce toit, tu t’y conformais. De l’autre côté, l’intelligence accompagnait la vie religieuse : les parents titillaient constamment nos esprits pour qu’on cherche le sens de ces gestes et de ces paroles. Et puis le fait d’avoir des ecclésiastiques à chaque génération démythologisait sainement l’institution. Quand on a des oncles ou des cousins, ou des frères ou des sœurs, consacrés à Dieu, on sait bien (parce qu’on connaît aussi leurs petits côtés !) que ce n’est pas parce que tu es curé ou bonne sœur que tu échappes à la commune humanité ! Et l’on considère les grandeurs d’établissement comme telles, et rien de plus. Il y avait donc le rite, rassurant et essentiel, et l’esprit critique, galvanisant.  Pourquoi commencer par là ?  Mon éditrice voulait que ce que j’ai à dire sur la vie à Jérusalem pût être entendu depuis sa source. D’où est-ce que je parle ? De cette alliance entre la foi et l’intelligence critique. On a urgemment besoin de la redécouvrir, non seulement dans le monde en général, mais aussi dans l’Église catholique aujourd’hui, où trop souvent on se réfugie dans des arguments d’autorité hiérarchique (les plus faibles qui soient), des logiques d’identité, ou derrière des influenceurs-gourous. J’adresse ce message aux (futurs) parents : le plus beau cadeau que vous puissiez faire à vos enfants c’est de prier ensemble d’abord, avec simplicité, et de susciter en eux la curiosité de poser toutes les questions, même les plus impertinentes !  Vous faites ensuite entrer le lecteur dans Jérusalem par les sons et les senteurs, racontés en prose poétique. Pourquoi cette description sensible avant de parler du « désastre » qui se déroule en Terre Sainte ?  Alors que tout le monde se tait sur Jérusalem, soit par horreur, soit par crainte de mettre de l’huile sur le feu, le défi était de trouver des mots qui soient plutôt de l’huile dans les engrenages, tout en disant les choses. Pour commencer l’ouvrage, l’approche par la beauté me semblait donc importante, car c’est une expérience partageable avec tout le monde, une expérience sensible. C’est très « thomiste », d’ailleurs : notre frère saint Thomas d’Aquin enseigne que rien ne vient à l’esprit humain qui ne passe d’abord par les sens.  Le même Thomas d’Aquin enseigne aussi qu’au fond, à échelle humaine, personne n’enseigne vraiment personne. Tout ce que nous pouvons faire c’est présenter des signes, dire des mots. Ce qui fait qu’ils ont un sens, c’est qu’ils résonnent dans le cœur de nos interlocuteurs habité par le « maître intérieur », cette étincelle de l’intelligence divine qui est donnée à chacun. C’est vraiment le centre de mon livre : la foi qui me reste est la conviction que le Christ est présent à toute personne humaine du simple fait qu’elle est dotée d’une conscience habitée par une parole.  Croire que le Christ est le Verbe divin en personne venu dans la chair, selon le dogme catholique, est très libérateur : cela invite à chercher la vérité en respectant intégralement la conscience d’autrui, car elle aussi participe de ce Verbe qui nous devance tous deux, qui est une personne divine et que par conséquent ni moi ni mon interlocuteur ne possédons, car personne ne possède personne, Dieu merci !  Après l’évocation poétique de Jérusalem, vous abordez le « désastre physique et moral » qui sévit en Terre sainte. Que peut-on en dire ?  Constatons d’abord les faits : une folie homicide semble s’être emparée des responsables des deux parties en présence. Thomas, un ami juif, chanteur d’opéra, dont la famille a beaucoup souffert par le passé en voyant la violence de la réaction israélienne à la violence déclenchée par le Hamas, n’avait plus de mots ; durant l’hiver 2023 il me disait : « Tout cela est tellement à l’opposé de tout ce pour quoi je vis… » Un autre ami, Emre, musulman soufi et avocat international à Istanbul, me disait récemment son effroi devant le degré d’inhumanité qu’on atteint… D’un côté, les gens du Hamas qui commettent des atrocités contre des Juifs et s’en vantent en direct sur les réseaux sociaux, en utilisant le nom de Dieu. De l’autre côté, une guerre déclenchée en citant Isaïe et qui semble devenue une véritable guerre de vengeance, au vu des dizaines de milliers de victimes collatérales, et un prétexte pour intensifier l’écrasement des non-Israéliens en Cisjordanie…  Un dimanche après-midi, dans l’émission de France Culture animée par Marc Weitzmann, on parlait du « droit de la guerre ». Pas du droit international ni du droit humanitaire, avait-il prévenu en préambule. Ça allait être sinistre, car il s’agirait des calculs possibles concernant les victimes. La proposition d’une intervenante m’a beaucoup marqué. Elle disait qu’il y avait une chose sur laquelle tous devraient tomber d’accord : « L’enfant ne peut pas être intégré à de tels calculs. » Par définition, l’enfant n’est pour rien dans la guerre. Or, ils sont morts par milliers à Gaza. Dès décembre 2023, j’ai été ébranlé quand j’ai parlé à un ami israélien d’une vieille dame et de sa fille qui étaient venues lui porter secours, tuées dans la cour de notre église de Gaza, où opéraient des snipers israéliens, et qu’il m’a répondu : « But it’s war ! »   Dans votre livre, au fil des pages, la conversation se tourne « de plus en plus vers les amis juifs », on sent une grande proximité avec l’âme juive…  Oui, nous autres chrétiens ne prenons pas facilement la mesure de la blessure, de l’insécurité ontologique de l’âme juive. Récemment Pierre, grand écrivain français juif, m’a fait l’amitié d’animer un après-midi au sujet de mon livre, dans le salon d’une amie très distinguée. Avec une vraie tristesse, il m’a rappelé à quel point ce que les Juifs ont vécu le 7 Octobre, puis la déferlante antisémite dans des pays qui étaient normalement des pays refuges, les déracine de nouveau. Le naufrage de la classe politique israélienne quant au fait d’avancer vers la paix avec les Palestiniens, l’échec de Tsahal à empêcher le massacre, et maintenant l’affreuse attente de la libération des otages, ont intensifié l’angoisse. Que l’on retrouve dans les mots de David Grossman : « Tragiquement, Israël n’a pas réussi à guérir l’âme juive de sa blessure fondamentale, la sensation amère de ne pas se sentir chez soi dans le monde. » En filigrane, dans ces mots que Pierre nous citait, on aperçoit le rapport de l’âme juive avec Dieu : une sorte de combat spirituel dont on ne sort pas indemne, comme celui de Jacob avec l’ange en Genèse, 32. Or comme chrétiens nous pouvons communier à cette « blessure fondamentale ». Par mon roman familial, je connais bien l’attachement à la terre, l’importance de la patrie. Mais la patrie véritable est au Ciel : le Dieu vivant ne fait d’aucune terre un absolu, il a le sens du relatif. Comme le dit Jésus dans l’Évangile de Jean, on ne l’adore exclusivement « ni à Jérusalem, ni au mont Garizim », mais « en esprit et en vérité »…  Cette guerre qui fait rage en Terre sainte s’invite aussi ailleurs, notamment en France. Comment échapper à cette importation du conflit ?  Dans sociétés comme dans nos religions, il y a vraiment une critique à faire au nom de Dieu, non pas d’un « Dieu » que je posséderais, mais du Dieu connu comme inconnu, à la fois tout-puissant et « tout-faible », qui se propose aux consciences sans s’imposer.  Je crois qu’il faut lutter contre la réduction des religions à des idéologies violentes, et donc cultiver la vertu de religion, au singulier, qui les transcende toutes. Il s’agit de retrouver le sens de Dieu. Dieu relativise tout ce qui n’est pas lui-même ou l’être humain créé à son image. Il nous invite raisonnablement à aimer et non pas à mépriser.  L’athéisme de nos sociétés les plonge dans des logiques purement séculières : tôt ou tard on finit avec la loi du plus fort. Mais reprenons conscience du fait qu’en réalité on a tous un « dieu » : c’est ce à quoi on sacrifie tout le reste, au quotidien. Plutôt que de s’adorer soi-même, ou son confort, ou seulement ses proches, ou je ne sais quelle identité idolâtrée (toutes choses qui finissent par nous mettre en conflit avec autrui), ne vaudrait-il pas mieux s’ouvrir au mystère de la création et reconnaître une Bonté toute-puissante à la source de toute réalité ? Faute d’être sensibles à une origine commune bienveillante, nous finissons par classer les choses en noir ou en blanc, profitables ou non, et les gens en bons ou méchants sans nuances : si jamais tu compatis avec toutes les victimes, alors tu n’as « rien compris ! ». Alors on devient incapables de guérir du passé, de pardonner… Peut-être que, en reposant cette question dans le débat public : « Au fond, c’est quoi votre dieu ? Qu’est-ce qui vous fait bouger ? », on apporterait de la clarté.  En contexte de guerre, quel sens donner à votre titre, Il nous reste la foi ?  Je parle bien sûr de la foi qui inclut la raison critique. Le même Dieu donne la raison à l’être humain et se révèle à travers l’Écriture sainte. C’est pourquoi, dans le drame de la Terre sainte, quiconque se prétend religieux a le devoir de questionner son rapport à l’Écriture sainte, et les relations à la terre, à l’homme, à Dieu qui en résultent.  Par exemple, est-il digne de la raison et digne de Dieu de penser qu’une succession de mots humains, une portion de texte biblique, soit purement et simplement « la Parole de Dieu » ? Ni le judaïsme traditionnel ni l’Église catholique n’ont jamais dit cela ! Les textes sont divers, et relatifs aux circonstances historiques de leur élaboration : ce sont les supports d’une Parole qui les dépasse. Pour la découvrir, il faut les interpréter, et donc faire usage de la raison à l’intérieur même de la tradition. La lecture messianique guerrière des épopées bibliques que promeuvent le fondamentalisme chrétien et le sionisme religieux est aussi sotte que le fondamentalisme islamique. Pour l’amour de Dieu et des hommes, il faut oser en parler, c’est le défi de mon livre.  Devons-nous donc toujours espérer ?  L’espérance est un fait, elle est toujours déjà là ! Dans le beau recueil sur Michel Houellebecq et Dieu publié en 2022 par une collègue universitaire, j’ai proposé la catégorie de « pulsion théologale » en découvrant, sous la plume de l’auteur désabusé des Particules élémentaires, presque un dépit de constater qu’il y avait encore de l’espérance qui se glissait dans ses mots. Sa poésie, trop peu connue, témoigne de l’irréductibilité de cette pression du divin dans nos existences. Tant que nous vivons, le désir de faire confiance, d’espérer et d’aimer ne cesse de nous tarauder : à Pâques 2025, oserons-nous y déceler… le bon Dieu ? Opinions  La « démocratie » contre le peuple : la thèse audacieuse de Pierre Rosanvallon  CHRONIQUE - Dans un entretien au Monde, l’historien Pierre Rosanvallon en appelle à la censure d’un peuple en révolte. Cette censure, il l’appelle démocratie.   5 min • Mathieu Bock-Côté P ierre Rosanvallon est depuis près de cinquante ans une figure majeure de l’intelligentsia française, et c’est vers lui que Le Monde s’est tourné pour justifier l’exécution judiciaire de Marine Le Pen, et défendre les tribunaux qui ont mené la charge. Sa thèse tenait à peu près en une formule : la légitimité démocratique des magistrats est plus grande que celle des élus. Plusieurs ont pu s’étonner : l’éminent penseur considère donc vraiment que la démocratie a bien davantage à voir avec le droit et les juges qui l’interprètent qu’avec la souveraineté populaire, qu’il réduit à la forme d’un peuple arithmétique, addition de volontés individuelles arbitrairement rassemblées par un rituel usé, pouvant se dissoudre au rythme du mouvement des humeurs travaillées par les démagogues ? Rien de tout cela, toutefois, ne devrait nous surprendre, car Rosanvallon, présenté comme un grand théoricien de la démocratie, a consacré une partie importante de son œuvre à la détacher de la figure du peuple, qu’il a depuis longtemps décrété introuvable - c’est d’ailleurs pour cela qu’il a été consacré grand théoricien.  Revenons-y : compagnon de route de la deuxième gauche, et longtemps occupé à refonder intellectuellement l’État-providence, Pierre Rosanvallon, dès le début des années 1990, se lance dans une grande enquête sur l’histoire de la démocratie, qu’on dit alors triomphante, et dont la définition ne lui semble pourtant plus aller de soi. Son travail, documenté, fouillé, recherché, sans le moindre doute, repose toutefois sur une idée étonnante : le peuple, en démocratie, serait introuvable, insaisissable. Il faudrait certes postuler son existence, au nom du droit reconnu à la société de s’autoproduire, de s’auto-instituer, à condition de reconnaître immédiatement qu’il n’existerait pas vraiment, sauf à travers, le système de représentation qu’on met en place pour l’apercevoir, ou même le construire. Il se matérialiserait ainsi à travers la société civile organisée, les différentes autorités administratives, ou encore les tribunaux, censés incarner la continuité institutionnelle dans le temps. On pourrait y voir une forme de corporatisme de gauche. À sa manière, Rosanvallon a repris la logique argumentative de la contre-révolution, pour l’investir d’un contenu progressiste.  On retrouvait là, évidemment, le présupposé épistémologique de toutes les gauches : la société n’est qu’une « construction sociale ». La société n’est pas construite à partir de réalités anthropologiques, mais à partir d’une projection philosophique, presque d’un plan d’ingénieur, même si elle se construit ensuite au fil de temps, par couches de sédimentation institutionnelle successives. Le peuple n’est pas antérieur à sa mise en forme institutionnelle. On y revient : Rosanvallon a tout fait pour relativiser le moment électoral de la démocratie, comme s’il s’agissait de sa forme primitive, sans jamais parvenir à le congédier totalement. Le peuple électoral, soumis à toutes les manipulations, doit être bridé ou, du moins, neutralisé. Chaque fois qu’on le consulte, il peut surprendre, démanteler l’œuvre du temps. Il peut sortir des consensus dans lesquels on l’invite à se reconnaître, et qui sont censés correspondre à ce qu’on a autrefois osé appeler le cercle de la raison. Il en est de même du référendum, qui peut toujours servir, hypothétiquement, mais dont on limitera l’usage et la portée. Il permet aux mauvaises humeurs de s’exprimer, aux démagogues et aux agitateurs de rejoindre l’opinion. Le peuple tiendrait bien davantage dans le foisonnement institutionnel censé l’incarner que dans l’expression d’une volonté majoritaire peu convaincante, même vulgaire.  Il y a un autre oublié chez Rosanvallon : la définition culturelle, ou même identitaire, du peuple. C’est pourtant ce peuple qui, dans l’histoire, peut surgir pour former de temps en temps un nouvel État indépendant. C’est aussi ce peuple, que je nomme le peuple historique, qui, en Europe, se lève aujourd’hui contre l’immigration massive, effrayé à l’idée de devenir minoritaire chez lui. Sans surprise, il trouve lui aussi sur son chemin des démographes et des sociologues pour lui expliquer qu’il n’existe pas. De ce point de vue, il y a une convergence profonde entre le travail de Pierre Rosanvallon, celui de Patrick Boucheron et d’Hervé Le Bras. Le premier a nié le peuple démocratique, le second le peuple historique, le troisième le peuple démographique. Cela nous rappelle que les sciences sociales entendent moins révéler le réel que d’en confisquer la représentation, en construisant un écran pour empêcher de l’apercevoir. Derrière leur jargon se cache une volonté démiurgique, celle de décréter le réel ou de l’abolir.  Et pourtant, le peuple existe, et il accueille généralement assez mal la savante démonstration de son inexistence, et la poursuite d’une politique ouvertement anti-majoritaire. Alors il se révolte. C’est ce qu’on appelle le populisme. Nous entrons dans une période insurrectionnelle, qu’il faut mater. Rosanvallon constate ainsi que « la démonisation du populisme n’a désormais plus aucun effet ». C’est faux, mais l’aveu est intéressant : il y a eu diabolisation du contradicteur, et d’une part importante de la population, ce qui n’est pas conforme à l’éthique de la conversation civique. Il faut alors plus que jamais verrouiller le débat public. Citons-le : « Il faut donc instaurer une vigilance du langage et poursuivre sans relâche les voleurs de mots et les trafiquants d’idées ». C’est ce qui se passe lorsqu’on trace un cordon sanitaire contre les mauvais partis et qu’on multiplie les délits d’opinion pour empêcher les mauvaises idées de circuler. C’est ici que Rosanvallon retrouve la fonction stratégique qui est la sienne dans le dispositif intellectuel français : point de convergence entre la gauche radicale et l’extrême centre, il en appelle à la censure d’un peuple en révolte. Cette censure, il l’appelle démocratie. Opinions  La tribune de 50 universitaires en soutien à Fabrice Balanche  La violente campagne de dénigrement dont le géographe est victime est un signal d’alarme, s’inquiètent plus d’une cinquantaine d’universitaires* dans une lettre ouverte initiée par Xavier-Laurent Salvador et signée notamment par Jean-Michel Blanquer et Luc Ferry.   5 min • Tribune collective L’affaire survenue le premier avril 2025 sur le campus de Bron (université Lyon 2), visant le géographe Fabrice Balanche, est d’apparence simple, mais elle révèle un tournant inquiétant dans le rapport que les universités françaises entretiennent avec la liberté académique, la neutralité du service public et la gestion des conflits idéologiques contemporains.  Fabrice Balanche, maître de conférences habilité à diriger des recherches, spécialiste reconnu de la Syrie et des dynamiques géopolitiques du Moyen-Orient, a été la cible d’une violente campagne de dénigrement orchestrée par un groupe d’étudiants se présentant comme « militants antiracistes ». Lui reprochant un supposé soutien au régime syrien de Bachar el-Assad ainsi que des positions prétendument « islamophobes », ces étudiants ont exigé son éviction de l’université, allant jusqu’à publier des visuels diffamatoires et à perturber son enseignement.  La violence du procédé contraste avec la rigueur du travail académique de Fabrice Balanche, dont les publications sont reconnues dans la communauté scientifique internationale. Mais ce n’est pas ici la qualité du savoir qui est jugée : c’est sa conformité à un récit idéologique. La présence même de Balanche dans l’espace universitaire devient un affront aux yeux de militants pour qui l’université ne doit plus être un lieu de recherche libre, mais un instrument au service de luttes identitaires.  L’élément religieux n’est pas secondaire dans cette affaire : il est central. Durant le ramadan, le même groupe d’étudiants avait organisé dix dîners de rupture du jeûne sur le campus, en dehors de tout cadre cultuel officiel. Loin d’être de simples gestes de convivialité, ces iftars ont servi à inscrire visiblement et durablement une pratique religieuse dans l’espace universitaire, au mépris du principe de neutralité du service public. Il ne s’agissait pas tant d’exprimer une foi individuelle que de marquer un territoire idéologique et identitaire, prélude à une offensive plus politique.  Dans ce contexte, la mise en cause de Fabrice Balanche apparaît comme une manœuvre stratégique. Loin d’être spontanée, elle s’inscrit dans une logique de propagande souvent relayée en France par des milieux islamo-gauchistes. La campagne contre Fabrice Balanche visait donc à délégitimer un universitaire dont les travaux contredisent certains récits militants sur la Syrie, en le caricaturant comme un suppôt du régime, quitte à travestir ses analyses, à ignorer ses sources et à attaquer sa personne plutôt que ses idées.  Mais le plus grave n’est peut-être pas l’attaque elle-même. Le plus préoccupant est la réaction - ou plutôt l’absence de réaction - de l’institution universitaire. L’administration de Lyon 2 s’est montrée d’une extrême prudence, puis d’une complaisance glaçante. La présidente de l’université, Mme Isabelle von Bueltzingsloewen, au lieu de soutenir l’un de ses enseignants agressé dans l’exercice de ses fonctions, a choisi de remettre en question sa légitimité scientifique en contestant publiquement sa liberté académique. Ce geste marque une véritable rupture morale. Cet abandon d’un collègue constitue un acte d’indignité.  Dans un premier temps, aucune mise au point claire n’a été faite, aucune défense ferme de la liberté académique n’a été formulée. Il a fallu attendre la mobilisation d’universitaires français et étrangers, ainsi que la motion unanime de l’UFR Temps et territoires dénonçant une atteinte grave à la liberté académique, pour que l’institution reconnaisse enfin l’ampleur du problème. Mais, à ce stade, le mal est fait : le signal envoyé est que l’université recule quand il faudrait résister. Ce recul prend la forme d’un évitement, d’une faiblesse institutionnelle qui permet aux forces les plus radicales d’imposer leur loi sur le terrain universitaire. Plutôt que d’affirmer des principes clairs - liberté académique, neutralité de l’espace public, primauté du savoir sur l’idéologie -, l’université laisse s’installer un rapport de force où les plus bruyants, les plus agressifs, les plus aptes à instrumentaliser les émotions et les symboles dictent leur agenda.  Cette passivité n’est pas sans conséquences. Elle encourage la censure et l’autocensure. Elle incite les enseignants-chercheurs à éviter certains sujets, à euphémiser leurs propos, à contourner les zones sensibles, de peur d’être dénoncés, harcelés ou lâchés par leur propre institution. Elle transforme les universités en lieux de surveillance mutuelle, où la liberté de penser devient suspecte, où la complexité scientifique doit céder au simplisme militant.  Ce qui s’est passé à Bron n’est donc pas un simple incident : c’est un signal d’alarme. Nous ne laisserons pas l’un des nôtres affronter seul cette offensive. Nous nous tenons résolument aux côtés de Fabrice Balanche, à la fois pour le défendre publiquement, mais aussi pour l’assister concrètement sur le plan judiciaire. Nos collectifs sont mobilisés et disposent des ressources nécessaires pour faire valoir le droit, la justice et l’honneur du service public. L’université française doit se ressaisir. Elle doit redire fermement que la recherche n’est pas un crime, que l’analyse critique n’est pas une agression, que la rigueur scientifique ne doit pas céder aux passions identitaires. Enfin, il faut tirer toutes les conséquences de cette crise : la présidente de l’université Lyon 2 ne peut pas rester en fonction après un tel reniement des principes fondamentaux de la république universitaire. Nous demandons sa démission.  * Retrouvez la liste des signataires sur lefigaro.fr Une-ECO  Quand les internautes chinois se moquent des États-Unis de Trump   1 min D es rangées d’ouvriers américains en surpoids, confus, s’emmêlant les doigts dans des machines à coudre : l’image, générée par intelligence artificielle, tourne en dérision les États-Unis, contraints de produire eux-mêmes leurs biens de consommation en raison de la guerre commerciale. Dans une autre vidéo, ce sont Donald Trump, son vice-président JD Vance et Elon Musk qui s’échinent sur des chaînes d’assemblage de chaussures ou d’iPhone.  Sur l’internet chinois, très contrôlé, les publications sarcastiques et humoristiques se sont multipliées à la suite de l’imposition par Washington de surtaxes colossales sur les produits chinois. Les droits de douane supplémentaires sur ces derniers peuvent atteindre jusqu’à 145 %, alors que Donald Trump a suspendu l’escalade de ses taxes douanières pour le reste du globe. La Chine a répliqué en instaurant des taux majorés de 125 points sur les produits américains et affirme vouloir mener ce bras de fer commercial « jusqu’au bout ».  En écho à la rhétorique officielle, les publications moquant la dépendance américaine aux produits chinois ont donc inondé les réseaux sociaux. Elles cumulent des millions de vues sur l’application de vidéos TikTok, bloquée en Chine par les autorités mais qui reste accessible via un réseau privé virtuel (VPN). Mais, derrière l’humour, se cache peut-être une vive inquiétude face à l’impact de la guerre commerciale sur l’économie chinoise, dépendante de ses exportations. J. G. Une-ECO  L’État investit 400 millions pour décarboner l’industrie   1 min • Danièle Guinot M algré les contraintes budgétaires, l’État continue à investir dans la décarbonation de l’industrie. La France va ainsi financer pour environ 400 millions d’euros de nouveaux projets, ont annoncé vendredi les ministères de l’Économie et de l’Industrie. Des appels à projets seront émis dans le cadre de France 2030, un plan d’investissement de 54 milliards sur cinq ans, lancé en 2021 pour soutenir des entreprises de secteurs clés et les aider à « rattraper le retard industriel », et « soutenir la transition écologique ». « Le gouvernement a fait de la maîtrise des finances publiques une priorité, mais cela n’écarte pas la menace climatique », a justifié Bercy, rappelant l’engagement de la France à parvenir à la neutralité carbone en 2050.   Les industriels qui répondront aux appels à projets pourront notamment bénéficier de subventions pour remplacer la chaufferie au gaz de leur usine par une autre utilisant la biomasse, d’origine végétale ou animale. Seront également financés des travaux préparatoires pour aider les usines de grandes zones industrielles, comme à Dunkerque, à coordonner leurs actions de décarbonation, par exemple avec des réseaux de chaleur. Des appels à projets seront aussi réservés aux petites entreprises, pour des « petites victoires rapides de décarbonation », comme la construction d’une canalisation pour se raccorder à un réseau de chaleur.   Après une accélération en 2023, la France a marqué le pas dans la décarbonation, avec seulement 1,8 % de baisse des émissions de gaz à effet de serre en 2024. D. Q. avec AFP Une-ECO  Emploi : Bercy va recevoir les syndicats chaque semaine   1 min • Pierre-Loeiz Thomas L es « enflammades » de Donald Trump sur les droits de douane pèsent sur l’emploi. Dans ce contexte de guerre commerciale, les partenaires sociaux et les ministères de l’Économie et du Travail se réuniront chaque semaine à Bercy pour « suivre comme le lait sur le feu la situation dans les entreprises ». « Nous avons gagné, il a fallu se battre, la mise en place d’une cellule de crise », s’est réjoui la secrétaire générale de la CGT, Sophie Binet, à l’issue d’une rencontre sur le sujet entre les syndicats et l’exécutif. « Aujourd’hui, il y a de très grosses inquiétudes de la part des salariés : dans les entreprises, il y a des suppressions de CDD, des missions d’intérim qui s’arrêtent, des contrats qui ne sont pas renouvelés et des plans sociaux qui continuent, avec, pour certaines entreprises, un effet d’aubaine », a avancé Marylise Léon, secrétaire générale de la CFDT.  Avec ce rendez-vous hebdomadaire impliquant le patronat, les syndicats espèrent endiguer l’érosion du marché de l’emploi, déjà malmené par l’incertitude politique. Selon la dernière enquête de France Travail sur les besoins en main-d’œuvre, le nombre de projets de recrutement pour 2025 a déjà chuté de 12,5 % par rapport à 2024. Cette pénurie d’offres devrait aller de pair avec un nombre de demandeurs d’emploi en augmentation. Dans sa note de conjoncture du mois de mars, l’Insee prévoit le rebond du taux de chômage de 0,2 point au premier trimestre 2025. PIERRE-LOEIZ THOMAS économie  Les placements refuges pour échapper à la guerre commerciale  Entre la hausse des droits de douane, le conflit en Ukraine et la dette française, les particuliers ont des raisons d’être prudents. Bonne nouvelle, les produits d’épargne peu risqués rémunèrent toujours bien.   8 min • Jorge Carasso Q ue faire avec son bas de laine en ces temps chahutés ? La guerre commerciale que livre Donald Trump au reste du monde a fait vaciller les marchés financiers (- 12 % pour le S&P 500 et le CAC 40 en cinq jours début avril). La guerre aux portes de l’Europe, menée par la Russie contre l’Ukraine, nourrit aussi les inquiétudes, tout comme l’instabilité politique française et le montant colossal de la dette hexagonale, par beaucoup jugé intenable. De quoi pousser les épargnant à chercher davantage de sécurité dans leurs placements ? « On sent de l’inquiétude, confirme David Charlet, président de l’Anacofi, une des principales associations de conseillers en gestion de patrimoine (CGP). Cela réactive de vieux réflexes sécuritaires, alors que ces derniers temps les épargnants avaient pris l’habitude de prendre un peu plus de risques, notamment sur les marchés. » Le krach d’avril a tout changé. « Quand des baisses d’une telle ampleur se produisent, les ménages ont tendance à être attentistes », confirme Éric Buffandeau, directeur adjoint des études chez BPCE (Banque populaires, Caisse d’épargne). Ce contexte pourrait être à nouveau favorable aux produits de taux - livrets, comptes à terme, fonds en euros de l’assurance-vie… -, à l’or, ou, de façon générale, aux produits peu risqués (obligations, monétaire). Bonne nouvelle, ces produits longtemps délaissés rapportent à nouveau.  L’or culmine à des sommets jamais atteints  L’éternelle valeur refuge n’a jamais coûté aussi cher. Dopé par l’appétit des banques centrales pour le métal jaune, la crainte d’un retour de l’inflation en cas de guerre commerciale ou encore la chute du dollar, le cours de l’once dépasse désormais les 3 300 dollars (+ 40 % en un an), un record. Mais, surtout, ce n’est peut-être pas fini. Les analystes ne cessent de rehausser leurs objectifs. Ceux de Goldman Sachs voient même la relique barbare caracoler à 4 000 dollars l’once l’an prochain. L’or est de loin l’actif qui tire le mieux son épingle du jeu en cette période troublée. Il est de bon ton d’en détenir un peu, quelle que soit d’ailleurs son appétence au risque. « On met toujours une pincée d’or dans le portefeuille actions de nos clients », confirme Éric Gérard, gérant associé chez Optigestion, une société de gestion. Les épargnants qui souhaitent acheter le précieux métal peuvent opter pour de l’or physique (pièces, lingotins, lingots…) ou sous forme d’investissements financiers (des fonds qui répliquent l’évolution des cours), à l’intérieur d’un contrat d’assurance-vie ou d’un compte-titres. « Toutes les compagnies d’assurance-vie ne l’acceptent pas », nuance toutefois Éric Franceschini, directeur de la clientèle privée chez Bordier et Cie. Surtout, l’or n’est pas une assurance tout risque. « C’est un placement intéressant, mais qui est volatil, rappelle Rony Michaly, directeur général de Galilée AM, une société de gestion. Etil navigue en outre à des niveaux déjà élevés. » En outre, l’or n’offre pas de rendement.  Les rendements retrouvés du fonds en euros  Longtemps mal aimé car trop peu rentable, le fonds en euros (sans risque) de l’assurance-vie reprend des couleurs. Dans un contexte de taux d’intérêt à nouveau élevés, les assureurs se livrent désormais une concurrence féroce pour attirer les épargnants dans leurs filets. Résultat, ils offrent des rendements plus qu’appétissants, parfois au-delà de 4 % grâce à leur politique de bonus (par exemple, + 2 % de rendement par an pour les nouveaux versements). « Ce sont le plus souvent des offres conditionnées à la souscription complémentaire de contrats en unités de compte (non garantis en capital) », rappelle Éric Coudert, président de la Financière de la Seine, un cabinet de CGP. L’an dernier, hors bonus, les fonds en euros ont rapporté, en moyenne, 2,60 %, selon l’association France Assureurs. De quoi séduire quand la Bourse fait le yo-yo. « Quand on est dans une période où il est difficile de voir où l’on va, le fonds euros a de bonnes cartes à jouer, confirme Élodie Rouault, conseillère patrimoniale chez Althos Patrimoine, un cabinet de CGP. Cela permet au client de rester liquide sans prendre de risque (il peut vendre, et le coût fiscal est acceptable, NDLR). Cela lui offre la possibilité d’être réactif en Bourse quand il y a des opportunités d’investissement intéressantes. »  Le livret A moins flamboyant mais toujours apprécié  La baisse du taux du livret A à 2,4 % le 1er février (contre 3 % depuis 2023) aurait pu faire fuir les épargnants. Il n’en est rien. Le livret A a fait mieux en février (940 millions d’euros de collecte nette) qu’en janvier (350 millions d’euros). Il faut dire que l’ambiance générale y est pour beaucoup. « Le contexte mondial demeure anxiogène », indique Philippe Crevel, directeur du Cercle de l’épargne. De plus, le taux du livret A reste encore attractif. « On n’est pas à 0,5 %, comme ce fut le cas en 2020 », poursuit Philippe Crevel. La collecte est toutefois en retrait sur un an, après une année 2024 record. La rémunération de ce placement star de l’épargne, révisé deux fois par an, devrait baisser à nouveau au 1er août : en prenant en compte la moyenne de l’inflation et des taux interbancaires, il pourrait atterrir à 1,7 %. Un taux correct, pour un produit défiscalisé, sans risque et sans frais, mais qui n’est plus imbattable (le fonds en euros de l’assurance-vie fait mieux).  Les comptes à terme dans le vert, les livrets dans le rouge  Avec la baisse continue des taux de la BCE ces derniers mois, les livrets non réglementés ont vu leur rémunération largement chuter. « Ils offrent 0,8 % en moyenne », rappelle Philippe Crevel. Rien d’affriolant, donc. Les épargnants, pour qui ces produits font office de sas d’attente en cas de trésorerie excédentaire, leur préfèrent les comptes à terme, qui offrent un rendement fixe pour une durée déterminée (entre 2,5 % et 3 % sur 12 mois pour les meilleures offres). Ils permettent de conserver un excédent de liquidité à bon compte, en attendant une opportunité d’investissement. « Ces produits n’existaient quasiment plus quand les taux étaient bas », rappelle Sébastien Champion, responsable partenaires et solutions chez Swissquote Bank Europe. Ils ont retrouvé des couleurs en 2023 et lors d’une bonne partie de 2024, à un moment où ils rapportaient jusqu’à 4 % ou 5 % par an. « Depuis peu, ils sont davantage concurrencés par le fonds en euros », poursuit Sébastien Champion. Tout en restant attractif pour bloquer un bon taux sans risque.  Les fonds obligataires intéressants  La remontée récente des taux à long terme (à la différence de l’évolution des taux courts pilotés par les banques centrales) offre une fenêtre de tir aux épargnants qui cherchent à doper leur rémunération, sans prendre trop de risques. À ce titre, les fonds obligataires datés - constitués de dettes d’entreprises cotées souvent connues du grand public comme Renault, Loxam, Paprec… - ont leur carte à jouer : ils offrent une rémunération établie à l’avance (autour de 4 % pour des dettes avec une bonne notation) et sont remboursables au bout d’une durée fixée au préalable (2, 3, 5 ans). Ces produits sont accessibles via l’assurance-vie ou les comptes-titres. « Ces placements permettent de cristalliser un taux, explique Éric Franceschini, directeur de la clientèle privée chez Bordier et Cie. Lorsqu’on les souscrit aujourd’hui, on capture la hausse des taux obligataires à nouveau à l’œuvre, et on en conserve le rendement sur la durée. » Attention, le capital investi n’est pas garanti, car un ou plusieurs émetteurs de la dette - les fonds sont souvent constitués d’une centaine de lignes - peuvent faire faillite. Mais celles-ci restent extrêmement marginales dans un portefeuille diversifié. « À titre d’exemple, le taux de défaut cumulé des entreprises bien notées (par exemple, BBB) est de l’ordre de 1 % sur 5 ans », indique Thibault Destrés, responsable de la gestion obligataire chez Anaxis AM. Les épargnants qui préfèrent limiter le risque au maximum peuvent se tourner vers le monétaire - des obligations remboursables à très court terme. « On peut aller chercher de bonnes rentabilités sur cette classe d’actifs », confirme Thomas Potabès, responsable développement en région du groupe Raynes, un cabinet de CGP. Celles libellées en euros, qui ne s’exposent donc pas à un risque de change, naviguent aujourd’hui autour de 2,5 %. Pas si mal pour un placement liquide comportant très peu de risque. J. C. économie  Les techniques pour protéger son portefeuille boursier dans la tempête   5 min • Hervé Rousseau L a guerre commerciale engagée par Donald Trump a provoqué une tempête d’une rare violence sur l’ensemble des places boursières mondiales. « L’ampleur des mesures dévoilées le 2 avril nous a surpris, mais c’est surtout l’incertitude entretenue en permanence par le président américain qui inquiète les investisseurs », explique Alexandre Hezez, directeur des investissements à la Banque Richelieu.   À Wall Street, dès le lendemain, 2 500 milliards de dollars sont partis en fumée en une seule journée, soit l’équivalent de la capitalisation du CAC 40. La décision de suspendre pour trois mois l’essentiel des droits de douane, moins de 24 heures après leur mise en place, a permis aux marchés de reprendre un peu de hauteur. Mais le retour au calme n’est qu’apparent. L’ambiance reste fébrile en Bourse et les marchés toujours extrêmement nerveux, à la merci d’une nouvelle sortie inattendue du président des États-Unis. Le « Vix », mesure de la volatilité à Wall Street, aussi appelé « indice de la peur, » est toujours perché à plus de 30, soit deux fois plus haut qu’en début d’année.  Et si la Bourse de New York est la première victime de la politique de Washington, le CAC 40, riche en multinationales exportatrices, est aussi fortement affecté. Depuis son pic historique de la fin du mois de février dernier à plus de 8 257 points, l’indice phare de la Bourse de Paris abandonne toujours plus de 11,5 % ; il est désormais dans le rouge depuis le début de l’année.  Pour Raphaël Thuin, chez Tikehau, la politique menée par Donald Trump marque « un changement de régime pour les marchés et elle aura des conséquences à long terme ». Les professionnels recommandent toutefois de ne pas céder à la panique, avec le risque de vendre au pire moment et de louper les éclaircies. La performance d’un portefeuille dépend en effet en grande partie de quelques séances de forte hausse. Sur les 25 dernières années, un investisseur placé sur l’indice large MSCI World qui aurait raté les dix meilleures séances aurait vu ses gains réduits de moitié. Mais les réflexes ont la vie dure et, bien souvent les investisseurs individuels ont tendance à acheter lorsque les marchés sont en pleine forme et à prendre peur quand les choses tournent mal.  Pour autant, les gérants ne restent pas immobiles. Le moment est en effet bien choisi pour passer en revue son portefeuille. Le secteur de la défense fait pratiquement l’unanimité auprès des gérants. Le fabricant de munitions et de blindés allemand Rheinmetall a plus que doublé de valeur depuis le début de l’année, Thales grimpe de plus 80 % et Dassault Aviation s’adjuge plus de 50 %. Et, en dépit de ces beaux parcours, les analystes sont convaincus que les acteurs européens de la défense conservent du potentiel. « Ces entreprises vont bénéficier en priorité des importantes dépenses de réarmement que prévoient les États européens pour compenser la fin annoncée du parapluie de la défense américain », explique Alexandre Hezez, à la Banque Richelieu.   « Les entreprises de services et celles dont les chaînes d’approvisionnement et de production sont locales, y compris à l’étranger, semblent également en bonne position », estime un gérant. Les spécialistes des services aux collectivités comme Veolia ou des télécoms, avec Orange, devraient ainsi tirer leur épingle du jeu. Les géants de l’électrification des économies et des infrastructures devraient également profiter des vastes plans de relance prévus en Europe, avec notamment Legrand, Schneider Electric, Bouygues ou Engie.   Pour accompagner, l’effort budgétaire des gouvernements, la Banque centrale européenne (BCE), s’est engagée dans une campagne de baisse des taux d’intérêt. Jeudi, elle a encore réduit son principal taux directeur de 25 points de base, le portant à 2,25 %. Cette baisse des taux devrait naturellement profiter aux spécialistes de l’immobilier, promoteurs et foncières. Le dividende constitue également une solide corde de rappel lorsque les marchés tanguent. La récente chute des indices a mécaniquement propulsé les rendements. Au sein du CAC 40, parmi les champions du dividende, le titre Carrefour offre environ 7,5 %, Engie rapporte plus de 7 %, TotalEnergies 6,7 % et Orange 6,5 %.  Par prudence, les gérants empochent enfin une part de leurs plus-values sur les valeurs qui ont le plus grimpé ces dernières années. Ils se constituent un confortable matelas de cash en portefeuille afin de pouvoir profiter des creux de marché et réaliser, le moment venu, de bonnes affaires sur des titres injustement délaissés.  Comme il est pratiquement impossible de prévoir les sorties et les tweets intempestifs de Donald Trump et les retournements des Bourses, le bon sens consiste à investir régulièrement, par exemple chaque mois, une somme identique, et ce, quel que soit l’état du marché. La médaille a toutefois un revers. L’investissement programmé augmente sensiblement le coût du courtage. Il est recommandé de négocier avec sa banque, voire de se tourner vers la concurrence des courtiers en ligne ou encore d’utiliser les trackers dont les frais sont très serrés. Pour les investisseurs les plus aguerris, certaines techniques permettent de se couvrir contre les baisses : achat de puts (options de vente) sur les indices, ETF inverses qui montent lorsque les marchés baissent, ou encore produits structurés. H. R. économie  Inquiets de Trump, des riches Américains déplacent leur fortune en Suisse   4 min • Anne de Guigné C ette année, en plus d’une lumière étincelante sur le lac, le printemps a apporté quelques bonnes nouvelles sonnantes et trébuchantes sur les rives du Léman. Selon plusieurs témoignages, la place financière helvétique bénéfice depuis février d’un nouvel afflux de capitaux, issus des États-Unis. Inquiets de la politique erratique de Donald Trump, et notamment de l’évolution du dollar, des riches Américains viennent déposer depuis plusieurs semaines dans les coffres-forts suisses une part de leur fortune.  « Nous constatons un regain d’intérêt de la part des clients résidents en Amérique depuis quelques semaines, confirme Hippolyte de Weck, gestionnaire de fortune genevois à la tête de Forum Finance, une société agrée par le gendarme des marchés américains, la Securities and Exchange Commission (SEC). Soit des clients, avec qui nous travaillions déjà, nous confient plus de capitaux, soit des grandes fortunes qui n’ont pas encore diversifié leurs avoirs au sein de différentes places financières viennent chercher un autre style de gestion chez nous avec une exposition dans des marchés différents que ceux traditionnellement proposés par les banques américaines. »  La gestion américaine expose en effet peu ses clients aux marchés actions et obligations non américains. Ces derniers se retrouvent donc très sensibles aux fluctuations du dollar. Jusqu’il y a quelques mois, personne ne s’en plaignait. Les performances des actifs américains dépassant largement celles des autres régions du globe. Las, les guerres commerciales sont venues troubler ce bel ordre. « Nous observons une demande plus forte de clients existants ainsi que de nouveaux clients auprès de Pictet North America Advisers », a ainsi confirmé la banque Pictet à la télévision suisse RTS. Aujourd’hui, seules trois banques suisses - Vontobel, Julius Bär et donc Pictet - acceptent encore d’ouvrir des comptes à des clients américains qui ne se contentent pas de déposer leur salaire mais souhaitent investir sur les marchés.   Les autres ont renoncé à ces clients pour échapper aux tractations avec l’Administration américaine, qui exige désormais d’innombrables remontées de données fiscales au sujet des contribuables américains. Après des années de guérilla juridique menée par Washington contre les banques suisses, le Parlement de Berne avait ratifié en 2014 l’accord sur la loi Fatca qui prévoit un « échange semi-automatique d’informations » entre les institutions helvétiques et l’administration fiscale américaine. Depuis, aucune banque suisse ne peut plus s’abriter derrière la loi sur le secret bancaire pour protéger ses clients étrangers. Un accord similaire ayant été passé avec l’Union européenne.  Malgré ces évolutions et le spectaculaire effondrement de Credit Suisse en 2023, les flux américains témoignent de l’attrait persistant de la gestion de fortune suisse. « Notre système démocratique est parfois lent, mais il permet à toutes les opinions de s’exprimer et de converger en un consensus. Les réformes prennent du temps et le peuple souverain préfère les changements progressifs. C’est pourquoi lorsque des troubles surgissent dans un pays, la Suisse est vue comme un havre de sécurité, pour les personnes comme pour leurs actifs. Cela ne nous empêche pas de voter sur des idées assez extrêmes, mais celles-ci sont toujours largement rejetées », expliquait dans une tribune récente Grégoire Bordier, président de l’Association de banques privées suisses.  « Après les dernières législatives françaises, il y a eu également un intérêt de la part de résidents français. Nous avons vu arriver quelques nouveaux clients qui ont déposé entre 500 000 euros et 5 millions, confirme Hippolyte de Weck. Ils suivaient l’exemple de grandes fortunes françaises qui se sont pour la plupart déjà organisées pour diversifier leur patrimoine parmi différents prestataires financiers. »  Au-delà de la stabilité, les clients fortunés des banques suisses évoquent des questions de confidentialité. « En Europe, les banques sont des agents du fisc gratuit, le fisc a accès à toutes les transactions, en Suisse, sauf soupçons graves, les banquiers ne communiquent pas la teneur de transactions », raconte une source. Selon une étude de 2024 du cabinet d’audit Deloitte, la Suisse demeure ainsi la première place pour la gestion de fortune dans le monde. « Avec un volume d’actifs internationaux de 2 200 milliards de dollars, la Suisse reste le premier centre de comptabilisation au monde, écrivent les analystes. Toutefois, son avance s’amenuise peu à peu : la Suisse est en effet talonnée par le Royaume-Uni, qui arrive en deuxième position avec seulement 8 milliards de dollars d’écart ».   La place financière est essentielle à l’économie helvétique : elle fournit 5 % des emplois totaux, 9 % du produit intérieur brut et 12 % des recettes fiscales de la Confédération, des cantons et des communes. Les flux américains font alors figure d’excellente nouvelle après le choc de 2023 : l’effondrement de Credit Suisse avait provoqué de gigantesques sorties de flux de capitaux en l’espace de quelques jours.  A DE G économie  Donald Trump menace de limoger Jerome Powell, président de la Fed  La Maison-Blanche étudie cette possibilité, a fait savoir vendredi son principal conseiller. Le président américain reproche au gouverneur de ne pas avoir baissé les taux d’intérêt.   6 min • Pierre-Yves Dugua L es attaques de Donald Trump contre Jerome Powell, président de la Réserve fédérale américaine, ont repris de plus belle. « Je ne suis pas content de lui. Je lui ai fait savoir et si je veux qu’il parte, il partira vite fait, croyez-moi », a assuré jeudi le président américain dans le Bureau ovale, aux côtés de première ministre italienne, Giorgia Meloni. Un peu plus tôt, le locataire de la Maison-Blanche écrivait sur son réseau Truth Social : « Il est plus que temps que le mandat de Powell se termine », alors que le second mandat du banquier central doit s’achever en mai 2026.  Ce vendredi, Donald Trump et son gouvernement étudient la possibilité de limoger le président de la banque centrale américaine (Fed), Jerome Powell, a fait savoir le principal conseiller économique de la Maison-Blanche. « Le président et son équipe continueront à se pencher sur la question » de limoger Jerome Powell, a dit à des journalistes le directeur du Conseil économique national, Kevin Hassett.  Acte de défiance   On voit dans ces critiques une réaction aux propos tenus la veille par Jerome Powell, devant l’Economic Club de Chicago, selon lesquels les droits de douane promis par la Maison-Blanche, dépassant toutes les anticipations, auraient un impact négatif sur l’emploi, la croissance et pourraient bien relancer l’inflation. « Il est très probable que les droits de douane entraînent une hausse au moins temporaire de l’inflation. Les effets inflationnistes pourraient également être plus persistants », a-t-il souligné. Un message qui fait écho à l’opinion de l’immense majorité des économistes et des milieux d’affaires. Des déclarations perçues comme un acte de défiance par la Maison-Blanche.  À moins que Donald Trump, pour des raisons politiques et électorales, ait plus simplement décidé de faire porter dès aujourd’hui à Jerome Powell la responsabilité d’une récession qui semble de plus en plus vraisemblable. Le président estime en effet que la Fed aurait dû abaisser son taux directeur depuis longtemps et qu’en tardant de le faire, elle se conduit de manière irresponsable. Le banquier central reste de marbre face aux critiques et injonctions de la Maison-Blanche : « Nous ne nous laisserons jamais influencer par une quelconque pression politique, a-t-il ainsi déclaré. Les gens peuvent dire ce qu’ils veulent, c’est très bien. Ce n’est pas un problème. Mais nous ferons ce que nous faisons, strictement sans tenir compte des facteurs politiques ou de tout autre facteur extérieur. »  L’élément nouveau dans cette campagne engagée depuis des mois, d’abord par le candidat Donald Trump, puis par le président élu, et maintenant par le président en exercice, est la révélation par la presse que le chef de l’État américain a multiplié les consultations ces dernières semaines pour sonder ses conseillers sur la possibilité de renvoyer purement et simplement le patron de la Fed.  Certains conseillers de Donald Trump l’encouragent dans cette voie qui est pourtant juridiquement douteuse, et certainement très dangereuse pour la crédibilité de la politique monétaire américaine et la confiance des marchés. Jerome Powell lui-même, questionné il y a quelques jours sur ce point, estime que la loi le protège. Il entend se défendre devant les tribunaux dans le cas où Donald Trump mettrait sa menace à exécution.  Seule une malversation pourrait, selon le texte de la loi, autoriser la Maison-Blanche à démettre le patron de la Fed. Un désaccord sur la politique de taux avec un président populiste désireux d’élargir ses pouvoirs propres, ne saurait justifier un limogeage de la personne la plus écoutée par les marchés mondiaux.  Seulement voilà, Donald Trump vient de renvoyer plusieurs hauts fonctionnaires démocrates à la tête d’agences théoriquement indépendantes du pouvoir exécutif, comme la Federal Trade Commission (commission fédérale sur le commerce) qui veille à l’application du droit sur la consommation et contrôle les pratiques anticoncurrentielles, et le National Labor Relations Board (Conseil national des relations au travail), qui protège les droits des employés du secteur privé. Sans attendre que la Cour suprême tranche sur la constitutionnalité de ces sanctions, le président est aussi tenté d’appliquer à la tête de la Fed son interprétation très large des prérogatives du chef de l’exécutif.  Affaiblissement du dollar   Au moins une voix dans le cabinet présidentiel s’élève pour déconseiller le renvoi de Jerome Powell avant la fin de son mandat l’an prochain : il s’agit de celle de Scott Bessent, le secrétaire au Trésor. Ce dernier a rappelé il y a quelques jours que l’indépendance de la politique monétaire était un « coffret à bijoux » que les États-Unis ne devraient jamais mettre en danger. Instaurée depuis la présidence de Jimmy Carter, l’indépendance de la Fed a du reste été copiée dans le monde entier.  Les arguments constitutionnels ne sont pas les plus percutants dans cette affaire. Il est clair qu’en cas de contentieux judiciaire entre la Fed et la Maison-Blanche, la réaction des marchés serait immédiatement négative. Sans attendre l’issue du différend devant la justice, qui pourrait intervenir après de longues semaines, on assisterait à une vive remontée des rendements obligataires et à une chute des actions, car la fragilisation de la Fed et sa politisation seraient perçues comme un abandon de la lutte contre l’inflation et une fuite en avant. L’impact d’une destitution d’un patron de la Fed affecterait aussi le dollar, monnaie de réserve dans le monde entier. Nul doute que Scott Bessent ait rappelé ces évidences à l’impétueux président des États-Unis.  Pire, en cas de tentative de coup d’État de la Maison-Blanche sur la Fed, le comité monétaire de l’institution pourrait décider immédiatement de réélire le gouverneur Powell à son poste de président du Conseil des gouverneurs. Le patron de la Fed est en effet détenteur d’un double mandat, aux durées différentes : celui de gouverneur qui court sur 14 ans, et celui de président du Conseil des gouverneurs qui est de 4 ans.  Selon le Wall Street Journal, Kevin Warsh, ancien conseiller économique de George W. Bush, a lui-même été consulté par Donald Trump sur l’éventuel renvoi de Jerome Powell que Donald Trump avait lui-même choisi en 2017. Warsh, ancien gouverneur de la Fed, a également déconseillé au président de sanctionner Jerome Powell. L’économiste professeur à l’Université de Stanford est certes pressenti comme favori pour lui succéder. Mais il ne voudrait probablement pas y parvenir dans le contexte trouble d’une destitution politisée, assortie d’une crise de confiance financière. P-Y.D. économie  La stratégie périlleuse de Washington de « grand encerclement » de la Chine   4 min • Pierre-Yves Dugua D ans le chaos créé par les droits de douane massifs imposés à la planète entière, puis partiellement mais temporairement suspendus, Donald Trump a clairement désigné la Chine comme l’adversaire numéro un. C’est à ce jour le seul pays avec lequel aucune négociation n’a été engagée. C’est aussi, de loin, celui dont les biens sont le plus surtaxés puisque les droits atteignent 145 %, en plus de ce qui était déjà en place. C’est enfin le pays, aux ambitions militaires croissantes, allié de la Russie contre l’Ukraine, avec lequel les États-Unis affichent le déficit commercial le plus important, soit près de 300 milliards de dollars, et qui a battu un nouveau record l’an dernier.  Face à ce problème, Scott Bessent, secrétaire au Trésor qui est de fait négociateur de première ligne en matière de droits de douane, pousse la Maison-Blanche à créer une alliance réunissant d’autres nations visées à un moindre niveau par des sanctions commerciales américaines. Son objectif étant par une approche commune d’isoler le géant asiatique. Donald Trump est séduit par la démarche, sans pour autant la défendre publiquement de manière explicite. Avec des accords commerciaux, « nous pourrons approcher la Chine de manière groupée », a affirmé Scott Bessent.  Baptisée par certains « Grand Encerclement », la stratégie ambitieuse revêt des aspects différents suivant les pays. Elle touche à la fois les questions purement commerciales, mais aussi les questions militaires. Pour les nations géographiquement proches de la Chine, et dépendantes de ses richesses, comme le Vietnam, le Cambodge, la Thaïlande ou la Malaisie, il s’agit d’abord de conditionner des baisses de droits de douane américains à un abandon du « transhipment » (transbordement) qui consiste depuis quelques années à laisser la Chine assembler sur leur territoire des biens qui sont ensuite exportés vers les États-Unis. Phénomène qui s’est accéléré depuis le premier mandat de Donald Trump et sa première croisade protectionniste contre la Chine. Sur le papier, ces biens comme des panneaux solaires, ne sont pas « chinois » et donc échappent aux pires surtaxes américaines. Dans la pratique, ils démontrent comment Pékin trouve des voisins disposés à l’aider à contourner les sanctions américaines. Selon la presse américaine, les négociations entre Washington et ces nations nouvellement surtaxées traitent de ces conditions. Xi Jinping a lancé sa contre-offensive, le président chinois était en tournée cette semaine en Asie du Sud-Est, enchaînant la Malaisie, le Vietnam et le Cambodge, pour resserrer les liens avec ses voisins.  Pour les grands pays industrialisés, comme la Corée ou le Japon, le phénomène de transbordement n’est pas le cœur du problème. Il faudrait plutôt que Séoul et Tokyo commercent moins avec la Chine et fassent investir davantage leurs industries dans des usines implantées aux États-Unis. La chose n’est pas facile car le Japon, par exemple, échange aujourd’hui bien plus de biens avec la Chine qu’avec l’Amérique. Voilà qui explique probablement le silence de Ryosei Akazawa, négociateur commercial nippon à Washington cette semaine, au sortir de ses entretiens avec Donald Trump et tous ses conseillers économiques. La Maison-Blanche parle de « grands progrès » quand la délégation japonaise se montre de son côté beaucoup plus circonspecte. « Les discussions à venir ne seront pas faciles », a commenté le premier ministre japonais, Shigeru Ishiba, se disant prêt à rencontrer lui-même Donald Trump.  Autre dimension du « Grand Encerclement » : les questions militaires. La Chine dépense beaucoup d’argent pour construire des ports dans plusieurs pays d’Asie, notamment au Cambodge. Ces investissements ont pour but de créer durablement des bases navales pour appuyer l’expansionnisme chinois dans toute l’Asie. Washington voudrait y mettre un terme. Pour cela, il faut que les États-Unis procurent à ces nations en développement des avantages économiques et commerciaux alléchants. Là encore, ce n’est pas facile de rivaliser contre la proximité géographique.  Le rêve d’encerclement de la Chine qui anime Scott Bessent s’inspire de l’approche consentie par le Mexique. Le voisin et premier partenaire commercial des États-Unis a, semble-t-il, accepté de s’interdire le « transhipment » et a en outre interdit aux constructeurs automobiles chinois d’investir dans des usines au Mexique dans le but d’inonder ensuite les États-Unis et le Canada, notamment de voitures électriques, en profitant des accords de libre-échange.  Ironiquement, l’approche défendue par le secrétaire au Trésor ressemble à celle de Barack Obama. En négociant en 2016 le « Partenariat transpacifique » (TPP), le président américain de l’époque cherchait déjà à réduire les droits de douane applicables à onze partenaires des États-Unis dans la zone Asie-Pacifique, dont le Vietnam, Brunei, la Malaisie et Singapour. Dans la même optique d’isolement de la Chine. À peine arrivé à la Maison-Blanche, en 2017, Donald Trump dénonçait l’accord, jugeant qu’il était capable d’obtenir « un meilleur deal ». P-Y.D. Entreprises  Donald Trump lâche du lest sur les porte-conteneurs fabriqués en Chine  Le président américain accepte de réduire la hausse des taxes envisagée sur les bateaux conçus dans l’empire du Milieu.   4 min • Jean-Yves Guérin P romettre le pire pour amener les autres pays à d’énormes concessions. Puis assouplir certaines sanctions et reporter leur mise en application. On a découvert la méthode Trump avec son obsession de renforcer les barrières douanières. À l’exception de la Chine, qui n’a pas voulu venir à la table des négociations, le président des États-Unis a accordé le 9 avril à tous les pays un répit de 90 jours pendant lesquels leurs exportations aux États-Unis ne seront taxées que de 10 %. Une semaine plus tôt, il voulait punir l’Europe d’une hausse de 20 % des droits de douane. Bis repetita sur les taxes concernant les porte-conteneurs « made in China ».  Début avril, Donald Trump menaçait de faire payer 1,5 million de dollars chaque fois qu’un navire fabriqué en Chine faisait escale dans un port américain. Une façon, selon lui, de redonner de la vigueur à la construction navale américaine tombée en déshérence quand ce secteur se porte très bien dans l’empire du Milieu (50 % des porte-conteneurs fabriqués dans le monde). Jeudi, l’United States Trade Representative (USTR), qui est l’équivalent du ministère du Commerce, a fait un peu machine arrière. Dans les six prochains mois, rien ne changera. À partir du 14 octobre, les armateurs seront certes soumis à un rude traitement. Moins terrible cependant que ce qui leur était promis jusqu’ici.  Les aménagements concédés par les États-Unis n’ont rien de spectaculaire, mais allégeront la note qui s’annonçait, dans la première version du texte, massive pour les armateurs : 30 milliards de dollars. Avec beaucoup de risques que cela fasse grimper le prix des produits. Pour limiter la casse, les bateaux fabriqués en Chine paieront une seule fois la taxe d’entrée sur le territoire américain. Et, même s’ils effectuent plus de six rotations annuelles au pays de l’Oncle Sam, ce prélèvement ne sera exigé que six fois. De même, cette taxe ne sera pas perçue pour les bateaux arrivant vides et repartant chargés de produits agricoles américains (blé, soja…).  Malgré tout, la contribution demandée ne sera pas anecdotique : 50 dollars la tonne en 2025, puis 80 dollars les trois prochaines années pour des bateaux sortant des chantiers navals chinois et appartenant à un armateur chinois, comme Cosco. S’agissant de porte-conteneurs fabriqués dans l’empire du Milieu, mais appartenant à des non-Chinois, ce sera 18 euros la tonne avec 5 euros d’augmentation par an d’ici à 2028. Ce n’est pas par bonté d’âme que Trump a accepté d’en rabattre sur ses exigences. Les armateurs l’ont convaincu des conséquences néfastes de ses positions maximalistes. « Soit nous devrons revoir à la baisse notre réseau, soit nous devrons ajouter ce coût en plus », expliquait début mars Soren Toft, directeur général du leader mondial des porte-conteneurs, MSC. En clair, au lieu de desservir plusieurs ports sur la côte Ouest des États-Unis (Los Angeles, Long Beach, Oakland…), ses bateaux n’auraient plus fait qu’un stop dans les plus grosses infrastructures : Los Angeles et Long Beach. Avec, très vite, un risque d’engorgement dans ces ports. Plus globalement, même si elle se montre bravache, l’Administration Trump sait bien qu’elle manie de la dynamite en changeant les règles du jeu du transport maritime par lequel s’effectue 80 % du commerce international. « Les navires et le transport maritime sont essentiels à la sécurité économique américaine et à la libre circulation des échanges », réaffirme le représentant américain au Commerce, Jamieson Greer.  « Inverser la domination »   En revanche, ce gouvernement se montre toujours offensif pour redonner son lustre d’antan au transport maritime américain. « Les actions de l’Administration Trump commenceront à inverser la domination chinoise, à répondre aux menaces qui pèsent sur la chaîne d’approvisionnement américaine et à envoyer un signal de demande pour les navires construits aux États-Unis », affirme Jamieson Greer. L’USTR se dit prête à donner quelques coups de pouce dans des domaines ciblés : ainsi, tous les armateurs seront tenus de convoyer dans un délai de quatre ans 1 % des exportations américaines de GNL sur des bateaux construits aux États-Unis. Ce pourcentage passerait à 4 % d’ici à 2035.  Autre dispositif, cette fois en faveur des navires chargeant des poids lourds : les transporteurs étrangers bénéficieront du remboursement des taxes douanières s’ils commandent un navire construit aux États-Unis au cours des trois prochaines années. Enfin, les armateurs (Matson ou Seabord Marine) qui assurent le transport maritime domestique vers des territoires américains éloignés comme Hawaï ou dans les Caraïbes ne paieront pas les suppléments de taxes exigés dans les autres ports. Pas de quoi cependant apaiser les relations avec la Chine. « Nous exhortons les États-Unis à respecter les règles multilatérales et à cesser immédiatement les pratiques abusives », a réagi le ministère chinois des Affaires étrangères. J.-Y. G. END
